In [0]:
Load_saved = True

save_dir = '/tmp/ISAN/'
load_dir = '' #change this the the data dir 

In [0]:
import pandas as pd
import matplotlib

import math
import os.path
import time
from sklearn.cluster import KMeans 

import scipy as sp #.spatial.distance.cosine(u, v)
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import copy
import math
import os.path
import time


import numpy as np
import pickle
import pandas

from sklearn.decomposition import PCA


seaborn.set_style("whitegrid")

In [0]:
#utility functions
def normalize_logits(logits, axis=-1):
  logits = logits - np.max(logits, axis=axis, keepdims=True)
  logits -= np.log(np.exp(logits).sum(axis=axis, keepdims=True))
  return logits

def xentropy(p_lp, q_lp, axis=-1):
  p_lp = normalize_logits(p_lp)
  q_lp = normalize_logits(q_lp)
  return -(np.exp(p_lp) * q_lp).sum(axis=axis)

def entropy(p_lp, axis=-1):
  p_lp = normalize_logits(p_lp)
  return -(np.exp(p_lp) * p_lp).sum(axis=axis)

def kl(p_lp, q_lp, axis=-1):
  p_lp = normalize_logits(p_lp)
  q_lp = normalize_logits(q_lp)
  return (np.exp(p_lp) * (p_lp - q_lp)).sum(axis=axis)

In [0]:
#more utility
import string
chars = [" "]
for c in string.ascii_lowercase:
  chars.append(c)
print chars

pretty_chars = copy.copy(chars)
pretty_chars[0] = "_"

def get_chars(l):
  c_list = [chars[i] for i in l]
  return c_list


mapping = {}
for i in range(27):
  mapping[chars[i]] = i

def standard_figure(width = 9, height = 3.2):
  fig = plt.figure(figsize=(width, height), dpi=300) #im
  return fig

                            
def soft_max(x, t = 1):
  return np.exp(x*t) / np.sum( np.exp(x*t))

def make_step(h, input_val, target = None, multiplier = 1):
  h = np.dot( w_x[input_val], np.reshape( h, [-1,1] )) + np.reshape(b_ixn[input_val], [-1,1] )*multiplier
  output = np.dot( np.reshape(h, [1,-1]), w_out) + out_bias
  if not target is None:
    loss = sess.run( [train_loss_np], feed_dict={output_placeholder: output, target_placeholder:target})[0]
    return h, output, loss
  else:
    return h, output
  
def make_step_back(h, input_val):
  h = np.transpose( np.dot( np.linalg.inv( w_x[input_val]), np.reshape( h, [-1,1] )  - np.reshape(b_ixn[input_val], [-1,1] )) )
  prob = soft_max(h.dot(w_out))
  return h, prob[0][input_val]

In [0]:
true_num_nodes = 27 * 216**2 + 2 * 27 * 216 + 216 + 27
print true_num_nodes

In [0]:
#Parameters - set as required
input_size = output_size = 27
num = 1280000
num_nodes = 216
snapshop = "" #"100001"
hpvalues = {}
batch_size = 1

n_random = 10000  #number of random states to have


tf.reset_default_graph()

sess = tf.InteractiveSession()

input_placeholder = tf.placeholder(tf.float32, shape=[None, input_size])
target_placeholder = tf.placeholder(tf.float32, shape=[None, input_size])
state_placeholder = tf.placeholder(tf.float32, shape=[None, num_nodes])
output_placeholder = tf.placeholder(tf.float32, shape=[None, 27])


train_loss_np  = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        output_placeholder, target_placeholder) / tf.log(2.0) )


temperature = tf.placeholder(tf.float32, shape=[1])

In [0]:
with open(os.path.join(load_dir, "x_y_wx_bix_wo_hz_ob.pkl")) as pkl_file:
  x, y, w_x, b_ixn, w_out, h_zero, out_bias = pickle.load(pkl_file)

In [0]:
# CREATE PCA fit for figure 1

losses = []
count = 0
h  = h_zero
counter = 0
h_space = h
h_other = h
random_states = []
losses = []
for i in range(10000):
  h_in = h
  input_val = np.argmax(x[i])
  h, outpt_np, loss =  make_step(h, input_val, y[i])
  
  if i % 100 == 0:
    h  = h_zero
  if i %100 > 10:
    random_states.append(h[:,0])
    losses.append(loss)
print(np.mean(losses))

pca_dim = num_nodes
pca = PCA(n_components=pca_dim)
pca.fit(random_states[:])

print len(pca.explained_variance_ratio_)
pca_basis = pca.components_

In [0]:
#pca plots for different architectures
with open(os.path.join(load_dir, "gru_308_evratio.pkl")) as pkl_file:
  gru308_ev = pickle.load(pkl_file)
  print gru308_ev[:5], len(gru308_ev)
with open(os.path.join(load_dir, "gru_145_evratio.pkl")) as pkl_file:
  gru145_ev = pickle.load(pkl_file)
  print gru145_ev[:5], len(gru145_ev)
with open(os.path.join(load_dir, "rnn_256_evratio.pkl")) as pkl_file:
  rnn256_ev = pickle.load(pkl_file)
  print rnn256_ev[:5], len(rnn256_ev)
with open(os.path.join(load_dir, "rnn_116_evratio.pkl")) as pkl_file:
  rnn116_ev = pickle.load(pkl_file)
  print rnn116_ev[:5], len(rnn116_ev)
 
standard_figure(width = 3.5, height=2.7)
#fig = plt.figure(figsize=(11, height), dpi=300) #im
plt.semilogy(gru308_ev, label = "GRU-308")
plt.semilogy(gru145_ev, label = "GRU-145")
plt.semilogy(rnn256_ev, label = "RNN-256")
plt.semilogy(rnn116_ev, label = "RNN-116")
plt.semilogy(pca.explained_variance_ratio_, label = "ISAN-216")
plt.ylim(.5e-3, .2)
plt.xlim(0, 210)
plt.ylabel("Explained variance")
plt.xlabel("PCA dim")
plt.grid(False)
plt.legend()

plt.tight_layout()
plt.savefig(save_dir + "pca_ev_plot_narrow.pdf")

In [0]:

#Fig1 / Master_plot
string = " quarterly revenue "
#string = " a lit of "
zoom_idx = 11

fig = standard_figure()

history = []
history_cont = []
max_history = len(string)
out_history = np.ones((max_history, 27))
space_norms = np.zeros(max_history)
first_letter = np.zeros(max_history)

canvas = np.zeros((max_history*28, 2*max_history + 4))*np.nan
probabilities = np.zeros((max_history*28,1))
  
df_limit_history = pd.DataFrame()
h  = np.zeros((num_nodes,1))
#history.append(h)
df_limit_history = pd.DataFrame()

count = 0
for i in range(max_history):
  if i > 0:
    h_in = h
  else:
    h_in = np.zeros((num_nodes,1))
  input_val = mapping[string[i]]
  sum_of_hm1_cont = np.zeros((num_nodes,1))

  for k in range(len(history)):
    new_h = np.dot( w_x[input_val], np.reshape( history[k], [-1,1] ))
    history[k] = new_h
    sum_of_hm1_cont = sum_of_hm1_cont + new_h

  new_offset = np.reshape(b_ixn[input_val], [-1,1])
  history.append(new_offset)
  h = sum_of_hm1_cont +  new_offset
  
  history_cont = []
  for k in range(len(history)):
    new_cont = np.dot( np.reshape(history[k], [1,-1]), w_out)
    canvas[i*28 : (i) * 28 + 27,(k)*2:(k+1)*2] = np.reshape( new_cont, [-1, 1])
    history_cont.append( new_cont )
  space_norms[i] = np.linalg.norm( history_cont[0]) 
  if i > 0:
    first_letter[i] = np.linalg.norm( history_cont[1]) 
  outpt_np = np.dot( np.reshape(h, [1,-1]), w_out) + out_bias
  probabilities[i*28 : (i) * 28+27] = np.reshape( soft_max(outpt_np) ,[-1,1])
  
  canvas[i*28 : (i) * 28 + 27,-2:]  =  np.reshape(outpt_np, [-1,1])
  canvas[i*28 : (i) * 28 + 27,-4:-2]  = np.reshape(out_bias,[-1,1])
  
  if i+1 < len(string):
    true_char = string[i+1]
  else:
    true_char = ""
  loss = sess.run( [train_loss_np], feed_dict={output_placeholder: outpt_np, target_placeholder:y[i]})

  d = {"index" : i, "state": h, "input":input_val, "state_in":h_in, "delta":h-h_in, "loss": loss, "history":history[:], "output": outpt_np, "props": soft_max(outpt_np), "history_cont": history_cont[:], 
       "input_char":string[i], "predict_char":chars[np.argmax(outpt_np)], "true_char": true_char }
  df_limit_history = df_limit_history.append(d, ignore_index=True)



#fig = standard_figure()
fig = plt.figure(figsize=(15,5))

#(ax1, ax2) = plt.subplots(2, sharex=True)  
plt.subplot(2,1,1)    
ax  = plt.gca()
ax.patch.set_facecolor('white')
plt.imshow(np.transpose(canvas), 'jet', interpolation = 'nearest', vmin = -5, vmax = 10)
#plt.ylabel("$\kappa^{t}_{s}$",rotation = 'horizontal',fontsize =17 )
#plt.text(-15,+10, "$\kappa^{t}_{s}$",fontsize = 17 )
plt.text(-15,+15, "$s$",fontsize = 17 )
currentAxis = plt.gca()
currentAxis.add_patch(matplotlib.patches.Rectangle((28*zoom_idx-1,-1), 28, len(canvas[0])+1, edgecolor="red", fill = False, linewidth=3))

ax.annotate("",
              xy=(0.56, .13), xycoords='figure fraction',
              xytext=(0.49, .13), textcoords='figure fraction',
              arrowprops=dict(arrowstyle="->"),
            )
ax.annotate("",
              xy=(0.065, .7), xycoords='figure fraction',
              xytext=(0.065, .9), textcoords='figure fraction',
              arrowprops=dict(arrowstyle="->"),
            )

_=plt.xticks( np.arange(1,max_history*28,28)  -2 )
_=plt.yticks( np.arange(-1,-1,max_history) )
plt.axis('tight')

for i in range(max_history):
  val =  string[i]
  if val ==" ":
    val = "_"
  plt.text(i*28 +14, -2.5 , val)
plt.text(-20, -2.5 , 'input:')

#fig.colorbar(ax)

plt.subplot(2,1,2, sharex=ax )  
plt.ylabel("Predicted next letter")
_ = plt.stem( probabilities, markerfmt='') 
# _ = plt.stem( probabilities, '.' ) 
_ = plt.xticks( np.arange(-1,max_history*28,28),[])
_ = plt.yticks( np.arange(0,1.1,0.5) )


for i in range(max_history):
  val =   np.argmax(probabilities[i*28 : (i) *28 + 27])
  plt.text(i*28 + val + 1, probabilities[i*28 + val] -0.07 , pretty_chars[val])
  #for k in range(5):
  #  plt.text(i*28 +k*5, -0.1 , pretty_chars[5*k])
  #plt.text(i*28 +25, -0.1 , 'z')
  plt.text(i*28 +14, 1.3 , str(i) )
plt.text(-20, 1.3, 'step:')
plt.xlabel("$t$", fontsize = 17)
plt.tight_layout()
plt.ylim(0, 1.2)
#plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
ax = plt.gca()
ax.yaxis.grid(False)

#plt.savefig(save_dir + "master_plot.pdf")




In [0]:
#fig = standard_figure()
fig = plt.figure(figsize=(15,5))
canvas1 = canvas[:, ::2]
canvas1 = canvas1.reshape((-1, 28, canvas1.shape[-1]))
final_lp = canvas1[:, :-1, -1]
canvas1 = canvas1[:,:-1,:-1]
unigram_lp = canvas1[0,:,-1]

canvas2 = np.zeros((canvas1.shape[0], canvas1.shape[-1]-1))

letters = np.zeros(canvas2.shape)

  
aggregate_low = lambda i,j: j #from step j
aggregate_high = lambda i,j: j+1 #till step j
#aggregate_high = lambda i,j: i+1 #till step i

measure = 'norm'
measure = 'xentropy'
measure = 'rev_xentropy'
measure = 'rev_kl'
measure = 'kl'

measure = 'entropy'


for i in range(canvas1.shape[0]):
  cell = canvas1[i, :, :]
  kl_divs = np.zeros((cell.shape[-1]-1, ))
  first_err = -1
  for j in xrange(kl_divs.shape[0]):
    cum_log_probs = unigram_lp + cell[:,aggregate_low(i,j):aggregate_high(i,j)].sum(1)
    letters[i, j] = np.argmax(cum_log_probs)
    if measure == "kl":
      kl_divs[j] = kl(final_lp[i, :], cum_log_probs)
    elif measure == "rev_kl":
      kl_divs[j] = kl(cum_log_probs, final_lp[i, :])
    elif measure == "norm":
      kl_divs[j] = np.linalg.norm(cum_log_probs)
    elif measure == "entropy":
      kl_divs[j] = entropy(cum_log_probs)
    elif measure == "xentropy":
      kl_divs[j] = xentropy(final_lp[i, :], cum_log_probs)
    elif measure == "rev_xentropy":
      kl_divs[j] = xentropy(cum_log_probs, final_lp[i, :])
    else:
      raise Exception("Unknown measure")
  kl_divs[i+1:] = np.nan
  canvas2[i, :] = kl_divs

plt.imshow(canvas2.T, 'Blues_r', interpolation='nearest', vmax=4)
plt.grid(False)
plt.axis('tight')
plt.xticks(*zip(*enumerate(string[1:])))
plt.yticks(*zip(*list(enumerate(string))))

if len(string)< 20:
  for i,j in zip(*np.nonzero(np.isfinite(canvas2))):
    if i==len(string)-1:
      continue
    if int(letters[i,j]) == mapping[string[i+1]]:
      color='k'
    else:
      color='r'
    #plt.text(i,j+0.3,pretty_chars[int(letters[i,j])], color=color)
plt.colorbar()

#plt.savefig(save_dir + "jan_entropy_plot.pdf")


In [0]:
#fig = plt.figure(figsize=(7, 7))

index = zoom_idx
hist = df_limit_history["history_cont"][index]
true_char = df_limit_history["true_char"][index]
#pred_char = df_limit_history["predict_char"][index]
pred_char = 'r'
pred_char_idx = mapping[pred_char]
true_char_idx = mapping[true_char]

standard_figure()
#fig = plt.figure(figsize=(11, 4), dpi=300)

plt.subplot(1,3,1)

p = plt.imshow(np.transpose(canvas[zoom_idx * 28: zoom_idx * 28 + 27, :]), 'jet', interpolation= 'nearest', vmin = -5, vmax = 10)

currentAxis = plt.gca()
currentAxis.add_patch(matplotlib.patches.Rectangle((pred_char_idx -0.5,-1), 1, len(string)*2+14, edgecolor="red", fill = False, linewidth=1))
currentAxis.add_patch(matplotlib.patches.Rectangle((true_char_idx -0.5,-1), 1, len(string)*2+14, edgecolor="orange", fill = False, linewidth=1))

plt.xticks(np.arange(0, 27), chars, fontsize = 8)
label_chars = df_limit_history["input_char"][:zoom_idx+1]
for i in xrange(len(label_chars)):
  if label_chars[i] == " " or label_chars[i] == "''":
    label_chars[i] = "_"
plt.title("a)")
#plt.yticks(2*np.arange(zoom_idx+1) + 0.5, label_chars)
plt.yticks( np.arange(-1,-1,max_history) )
plt.xlabel("Output character")
plt.text(-7, 10, "$\kappa_s^{11}$", fontsize = 17 )
plt.grid(False)
plt.colorbar(p)



print true_char_idx, true_char, pred_char

values = [hist[i][0][true_char_idx] for i in xrange(len(hist))]
values_pred = [hist[i][0][pred_char_idx] for i in xrange(len(hist))]

max_val = np.max([np.max(values), np.max(values_pred)]) + 0.5
min_val = np.min([np.min(values), np.min(values_pred)]) - 0.5

for i in xrange(len(values)):
  print df_limit_history["input_char"][i], values[i]

ind = np.arange(len(hist))    # the x locations for the groups
width = 1.0       # the width of the bars: can also be len(x) sequence

summed = np.zeros(27)
#positive_summed = np.zeros(27)

plt.subplot(1,3,2)

p = plt.bar(ind, values, width/2.5, color = 'orange', label = 'n logit')
p = plt.bar(ind + width/2.5, values_pred, width/2.5, color = 'red', label = 'r logit')

plt.title("b)")
#plt.title("Contribution to '{}' logit".format(true_char))
_ = plt.xticks( np.arange(0,len(hist),1) + 0.5, label_chars)
plt.ylim(min_val, max_val)
plt.grid(False)

plt.legend(loc=0)
height = max(values)

input_chars = [df_limit_history["input_char"][i] for i in xrange(len(hist))]

values = [hist[i][0][true_char_idx] for i in xrange(len(hist))]
values_pred = [hist[i][0][pred_char_idx] for i in xrange(len(hist))]


ind = np.arange(len(hist))    # the x locations for the groups
width = 1.0       # the width of the bars: can also be len(x) sequence


sum_idx1 = 7
true_char_sum1 = np.sum(values[:sum_idx1])
true_char_sum2 = np.sum(values[sum_idx1:])
pred_char_sum1 = np.sum(values_pred[:sum_idx1])
pred_char_sum2 = np.sum(values_pred[sum_idx1:])

p_values1 = [true_char_sum1, true_char_sum2, true_char_sum1+true_char_sum2]
p_values2 = [pred_char_sum1, pred_char_sum2, pred_char_sum1 + pred_char_sum2]

plot_ind1 = [1, 4, 7]
plot_ind2 = [2, 5, 8]

ax = plt.gca()

ax.annotate("",
              xy=(0.03, .4), xycoords='figure fraction',
              xytext=(0.03, .6), textcoords='figure fraction',
              arrowprops=dict(arrowstyle="->"),
            )

ax.annotate("",
              xy=(0.58, .06), xycoords='figure fraction',
              xytext=(0.51, .06), textcoords='figure fraction',
              arrowprops=dict(arrowstyle="->"),
            )
plt.xlabel("s       ")

plt.subplot(1,3,3)

p = plt.bar(plot_ind1, p_values1, width, color = 'orange', label = "n logit")
p = plt.bar(plot_ind2, p_values2, width, color = 'red', label =  'r logit')
plt.title("c)")
plt.legend(loc =0)
#plt.title("Contribution to '{}' logit".format(pred_char))
plt.grid(False)
_ = plt.xticks( [1.5,3, 4.2,5.5, 7.5], ["_annual", "+", "_reve", "=", "_annual_reve"])
plt.text(3.3, 1.5, "+")
plt.text(6.2, 1.5, "=")
# plt.text(1.0, -1, "$\kappa_{\_annual}^{11}$")
# plt.text(4.5, -1, "$\kappa_{\_reve}^{11}$")
# plt.text(4.5, -1, "$\kappa_{\_reve}^{11}$")
#plt.ylim(min_val, max_val)
height = max(values)



# plt.tight_layout()  
plt.tight_layout(w_pad=-.2)  
#plt.savefig(save_dir + "zoom_in_plot.pdf")

In [0]:
#fig2 NORM OF ELEMENTS IN THE BUFFER
errors = np.zeros(50)
#h  = np.zeros((1, num_nodes))
h  = h_zero
losses = []
counter = 0
max_history = 200
history = [0 for i in xrange(max_history)]
input_history = [0 for i in xrange(max_history)]
df_limit_history = pd.DataFrame()

count = 0
for i in range(10000,15000):
  if i % 1000 == 0:
    print i, len(df_limit_history)
  h_in = h
  input_val = np.argmax(x[i])
  for k in range(min(max_history, count)):
    new_h = np.dot( w_x[input_val], np.reshape( history[k], [-1,1] ))
    out_val = out_val + new_h
    history[k] = new_h
    
  out_val = np.zeros((num_nodes,1))
  for k in range(min(max_history, count)):
    if not k > max_history:
      out_val = out_val +  history[k]
  h = out_val +  np.reshape(b_ixn[input_val], [-1,1] ) 
  if len( history) == max_history: 
    history[-1] = history[-1] + history[-2]
    history.pop(-2)
    input_history.pop(-2)
  history.insert(0,np.reshape(b_ixn[input_val], [-1,1] ) )
  input_history.insert(0,input_val)
  outpt_np = np.dot( np.reshape(h, [1,-1]), w_out) + out_bias
  loss = sess.run( [train_loss_np], feed_dict={output_placeholder: outpt_np, target_placeholder:y[i]})
  count += 1
  if i  > 10:
    losses.append(loss)
    d = {"index" : i, "state": h, "input":np.argmax(x[i]), "state_in":h_in, "delta":h-h_in, "dist": i % 50, "loss": loss, "history":history[:], "input_history": input_history[:] }
    df_limit_history = df_limit_history.append(d, ignore_index=True)
print("produced {} random states".format(n_random))
print(np.mean(losses))


#dropping the most recent offset from the current prediction reduces accuracy to 1.9, droppping the 1st contribution to 1.718, 2nd 1.66771

In [0]:
#Calculate norm of various components in the buffer
vals = np.zeros((len(df_limit_history),max_history, 27))
logit_val = np.zeros((len(df_limit_history),max_history, 27))
length = np.zeros((len(df_limit_history),max_history, 27))
logit_length = np.zeros((len(df_limit_history),max_history, 27))
for i in range(len(df_limit_history)):
  i_vec = np.zeros((num_nodes,1))
  for j in range( min(max_history, i)-1, -1, -1):
    save_dex = df_limit_history["input_history"][i][j]
    vals[i,j, save_dex] = np.linalg.norm(df_limit_history["history"][i][j])
    i_vec = i_vec + df_limit_history["history"][i][j]
    length[i,j, save_dex] = np.linalg.norm(i_vec)
    logit_length[i,j, save_dex] = np.linalg.norm(np.dot( np.reshape(i_vec, [1,-1]), w_out) )
    logit_val[i,j, save_dex] = np.linalg.norm( np.dot(np.reshape(df_limit_history["history"][i][j], [1,-1]), w_out))


In [0]:
#FIGURE DECAY AND IMPORTANCE STARTS HERE
#spacebar #Fig5 #importance of space bar

#get weights
fig = plt.figure(figsize=(20, 15))
df_limit_history_space = pd.DataFrame()
losses = []
count = 0
h  = h_zero
counter = 0
h_space = h
h_other = h
for i in range(10000):
  h_in = h
  input_val = np.argmax(x[i])
  if input_val == 0:
    multiplier_space = 1
    multiplier_other = 0
    counter = 0
  else:    
    multiplier_space = 0
    multiplier_other = 1    
    counter = counter + 1
  h_space, outpt_np_space, loss_space = make_step(h_space, input_val, y[i], multiplier_space)
  h_other, outpt_np_other, loss_other = make_step(h_other, input_val, y[i], multiplier_other)
  h, outpt_np, loss =  make_step(h, input_val, y[i])

  d = {"index" : i, "input":input_val, "state_in":h_in, "counter":counter, "loss": loss, "loss_space": loss_space, "loss_other": loss_other, "predict_char":chars[np.argmax(outpt_np)], "true_char": true_char }
  df_limit_history_space = df_limit_history_space.append(d, ignore_index=True)

  if i > 10:
    losses.append(loss)
print(np.mean(losses))

space_only = df_limit_history_space.groupby(['counter'])["loss_space"].median().values
without_space = df_limit_history_space.groupby(['counter'])["loss_other"].median().values
all_chars = df_limit_history_space.groupby(['counter'])["loss"].median().values

In [0]:
#norm_decay
fig =standard_figure()
#plt.plot(length.mean(0))
x_lim = 20

x_axis = np.arange(x_lim+1)

plt.subplot(1,3,1)
#plt.title("Norm of logit contribution")
plt.title("a)")
for i in range(27):
  normalising = np.sum(( logit_val[:,:x_lim+1,i]  != 0), 0 )
  y_s =  (logit_val[:,:,i].sum(0)[:x_lim+1]) /normalising
  if i < 5:
    plt.semilogy(x_axis, y_s, label = pretty_chars[i])
  elif i ==5:
    plt.semilogy(x_axis, y_s, label = "...")
  else:
    plt.semilogy(x_axis, y_s)
    # plt.semilogy(x_axis, (logit_val[:,:,0].mean(0)[:x_lim]), 'g', label = 'space')
# plt.semilogy(x_axis, (logit_val[:,:,1].mean(0)[:x_lim]),'r', label = 'not space')
plt.xlabel('Number of steps')
plt.ylabel('Norm of logit vector')
plt.legend()

plt.subplot(1,3,2)
#plt.title("Norm of logit contribution")
plt.title("b)")
temp = np.sum(logit_val,2)
plt.semilogy(temp.mean(0)[:-100], 'g', label = 'all chars')
#plt.loglog((logit_val[:,:,1].mean(0)[:-1]),'r', label = 'not space')
plt.xlabel('Number of steps')
#plt.ylabel('norm of logit vector')
plt.legend()

plt.subplot(1,3,3)

plt.plot(without_space[:14], label = 'no space')
plt.plot(space_only[:14], 'g', label = 'space only')
plt.plot(all_chars[:14], 'r', label = 'all')
plt.xlabel('Position in word')
plt.ylabel('Median perplexity')
#plt.title('Perplexity by position in word')
plt.title("c)")

plt.legend()


plt.tight_layout()
plt.savefig(save_dir + "norm_decay.pdf")

In [0]:
#Fig4 / Master_plot by words
#string = " the united states of america caused a lot of conflict "
string = " the annual revenue was higher than expected "
n_words = len(string.split(" "))-1
fig = standard_figure()

history = []
history_cont = []
max_history = len(string)
out_history = np.ones((max_history, 27))
space_norms = np.zeros(max_history)
first_letter = np.zeros(max_history)

canvas = np.zeros((max_history*28, 2*n_words + 4))
probabilities = np.zeros((max_history*28,1))
  
df_limit_history = pd.DataFrame()
h  = np.zeros((num_nodes,1))
#history.append(h)
df_limit_history = pd.DataFrame()

range_temp =  len( canvas[1,:])/2
superprobcanvas = np.zeros((max_history,range_temp))


count = 0
for i in range(max_history):
  if i > 0:
    h_in = h
  else:
    h_in = np.zeros((num_nodes,1))
  input_val = mapping[string[i]]
  sum_of_hm1_cont = np.zeros((num_nodes,1))
  sum_of_hm1_cont_super =  np.reshape(b_ixn[input_val], [-1,1])
  for k in range(len(history)-1, -1, -1):
    new_h = np.dot( w_x[input_val], np.reshape( history[k], [-1,1] ))
    history[k] = new_h
    sum_of_hm1_cont = sum_of_hm1_cont + new_h
    
    sum_of_hm1_cont_super = sum_of_hm1_cont_super + np.reshape(new_h, [-1,1])
    if i < (len(string) - 1):
      temp = np.dot( np.reshape(sum_of_hm1_cont_super, [1,-1]), w_out) + out_bias
      superprobcanvas[i, k] = np.reshape( soft_max(temp ) ,[-1])[mapping[string[i+1]]]
      #print sum_of_hm1_cont_super.sum()
      #print     superprobcanvas[i, k]
    

  new_offset = np.reshape(b_ixn[input_val], [-1,1])
  if input_val == 0:
    history.append(new_offset)
  else:
    history[-1] = history[-1]+new_offset
  h = sum_of_hm1_cont +  new_offset
  
  history_cont = []
  for k in range(len(history)):
    new_cont = np.dot( np.reshape(history[k], [1,-1]), w_out)
    canvas[i*28 : (i) * 28 + 27,(k)*2:(k+1)*2] = np.reshape( new_cont, [-1, 1])
    history_cont.append( new_cont )
  space_norms[i] = np.linalg.norm( history_cont[0]) 
  outpt_np = np.dot( np.reshape(h, [1,-1]), w_out) + out_bias
  probabilities[i*28 : (i) * 28+27] = np.reshape( soft_max(outpt_np) ,[-1,1])
  
  canvas[i*28 : (i) * 28 + 27,-2:]  =  np.reshape(outpt_np, [-1,1])
  canvas[i*28 : (i) * 28 + 27,-4:-2]  = np.reshape(out_bias,[-1,1])
  
  if i+1 < len(string):
    true_char = string[i+1]
  else:
    true_char = ""
  loss = sess.run( [train_loss_np], feed_dict={output_placeholder: outpt_np, target_placeholder:y[i]})

  d = {"index" : i, "state": h, "input":input_val, "state_in":h_in, "delta":h-h_in, "loss": loss, "history":history[:], "output": outpt_np, "props": soft_max(outpt_np), "history_cont": history_cont[:], 
       "input_char":string[i], "predict_char":chars[np.argmax(outpt_np)], "true_char": true_char }
  df_limit_history = df_limit_history.append(d, ignore_index=True)



fig = standard_figure()

supercanvas = np.zeros((max_history, range_temp))
superprob = np.zeros(max_history)
#superprobcanvas = np.zeros((max_history,range_temp))

for i in range(max_history):
  outpt_np = np.zeros((1,27))
  for k in range(range_temp):
    supercanvas[i,k] = np.linalg.norm(canvas[i*28 : (i) * 28 + 27,(k)*2:(k*2+1)])
    if k <  range_temp -1:
      outpt_np = outpt_np + canvas[i*28 : (i) * 28 + 27,(k)*2:(k*2+1)]
#     if i < len(string) - 1:
#       superprobcanvas[i, k ] = np.reshape( soft_max(outpt_np) ,[-1,1])[ mapping[string[i+1]]]
  if i < len(string) - 1:
    superprob[i] = probabilities[i*28 + mapping[string[i+1]]]
  
    
#(ax1, ax2) = plt.subplots(2, sharex=True)  
plt.subplot(2,1,1)    
ax  = plt.gca()
#plt.imshow(np.transpose(canvas), 'jet', interpolation = 'nearest', vmin = -5, vmax = 10)
plt.imshow(np.transpose(supercanvas), 'jet', interpolation = 'nearest', vmin = -5, vmax = 10)
#plt.ylabel("kappa per word")
plt.text(-4,+5, "$\kappa^{t}_{word}$",fontsize = 17 )

_=plt.xticks( np.arange(0,max_history,1) )
_=plt.yticks( np.arange(-1,-1,max_history) )
plt.axis('tight')

for i in range(max_history):
  val =  string[i]
  if val ==" ":
    val = "_"
  plt.text(i, -1.4, val)
plt.text(-3, -1.4, 'Input:')

#fig.colorbar(ax)

plt.subplot(2,1,2, sharex=ax )  
plt.ylabel("$P(\mathbf{x}_{t+1}$)", fontsize= 'medium')
_ = plt.xticks( np.arange(-0.5,max_history,1),[])
_ = plt.yticks( np.arange(0,1.1,0.5) )
plt.ylim(0,1.0)

for i in range(max_history):
  if i < len(string) -1:
    val =  string[i+1]
    if val ==" ":
      val = "_"
    plt.text(i, -0.15, val)

for i in range(  superprobcanvas.shape[1] ):
  _ = plt.plot( superprobcanvas[:-1,i],alpha = (1- i *1.0/superprobcanvas.shape[1]) , color = 'b' ) 
#plt.plot( superprobcanvas)  


# for i in range(max_history):
#   val =   np.argmax(probabilities[i*28 : (i) *28 + 27])
#   plt.text(i*28 + val-0.5, probabilities[i*28 + val] , pretty_chars[val])
#   number = 3
#   for k in range(number):
#     plt.text(i*28 +k*(27 // number), -0.1 , pretty_chars[(27 // number)*k]+ ",..")
#   plt.text(i*28 +25, -0.1 , 'z')
#   plt.text(i*28 +14, 1.1 , str(i) )
# plt.text(0, 1.1 , 'step:')


#plt.tight_layout()
#plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)


plt.savefig(save_dir + "master_plot_caused_conflict.pdf")


In [0]:
# SVD spectrum plot
#

standard_figure(width = 9, height = 4)

train_string = ''.join([pretty_chars[x[i].argmax(1)[0]] for i in range(len(x))]).replace('_', ' ')
hs = [h_zero.T]

for char in train_string:
  hs.append(make_step(hs[-1], mapping[char])[0])

hs = np.hstack(hs[1:])


u,s,v = np.linalg.svd(w_x[mapping['a']])

v0_acts = v[:1,:].dot(hs)
space_logs = (w_out.T.dot(hs) + out_bias[:,None])[mapping[' ']]

plt.subplot(1,2,1)

v0s = []

for i,w in enumerate(w_x):
  u,s,v = np.linalg.svd(w)
  v0s.append(v[:1,:].T)
  label=None
  color='blue'
  if i==0:
    label='space'
    color='red'
  elif i==1:
    label='a-z'
  plt.plot(s, color=color, label=label)
legend = plt.legend(loc='upper right', framealpha=1.)
legend.get_frame().set_facecolor('#FFFFFF')
legend.get_frame().set_alpha(1.0)
plt.title('Singular value spectrum')
plt.xlim(-10,225, )


plt.subplot(1,2,2)
space_logs = (w_out.T.dot(hs) + out_bias[:,None])[mapping[' ']]
plt.scatter(v0_acts, space_logs, alpha=0.1, marker='.')
#plt.axis('square')
plt.xlabel("Hidden states projected onto\nfirst singular vector of 'a'")
plt.ylabel("Unnormalized log prob.\nof space token")
plt.title("First singular vectors predict space token")
plt.text(1, 20, "$r=%.2f$" % (np.corrcoef(v0_acts, space_logs)[0,1]),
        bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))
np.corrcoef(v0_acts, space_logs)
# /plt.ylim(0, 1.2)


plt.tight_layout()

# save to png to not overload pdf readers with the pointcloud
plt.savefig(save_dir + "svd_spectrum.png",dpi=300)

#plt.savefig('./svd_spectrum.png',dpi=300)

In [0]:
#Basis changes
w_out_temp = np.transpose(w_out)
u,s,v = np.linalg.svd(w_out_temp)
temp = w_out_temp.dot(np.transpose(v))
b_1_inv = np.transpose(v)
b_1 = np.linalg.inv(b_1_inv)
q_1 = w_out_temp.dot(b_1_inv)[:27,:27]

block_1 = np.identity(num_nodes)
for i in range(27):
  q_1[i] = q_1[i] / np.linalg.norm(w_out_temp[i])
block_1[:27,:27] = q_1

plt.subplot(6,1,1)
plt.imshow(w_out_temp.dot(b_1_inv))

plt.subplot(6,1,2)
readout_basis = b_1 #block_1.dot(b_1)
plt.imshow(w_out_temp.dot(np.linalg.inv(readout_basis)))

w_out_readout = np.transpose(w_out_temp.dot(np.linalg.inv(readout_basis)))
w_out_pca = np.transpose(w_out_temp.dot(np.linalg.inv(pca_basis)))
b_ixn_pca =  np.transpose(np.dot(pca_basis, np.transpose(b_ixn)))
b_ixn_readout =  np.transpose(np.dot(readout_basis,  np.transpose(b_ixn)))
#pca.components_w_out_readout = np.transpose(w_out_temp.dot(np.linalg.inv(readout_basis)))


In [0]:
### UNIGRAM / BIGRAM PLOTS
#this cell builds the bigram_freq and cond_probs dicts. 
#bigram_freq[(char1, char2)] contains the counts of the bigram char1,char2 (both integers in [0,26]
#cond_probs[(char1, char2)] contains prob(char2 | char1)


freq = np.zeros(27)
for i in x:
  freq[np.argmax(i)] +=1
  
freq = freq / np.sum(freq)

from collections import Counter
bigram_freq = Counter()

for i in xrange(len(x)-1):
  bigram_freq[(np.argmax(x[i]), np.argmax(x[i+1]))] +=1

cond_probs = Counter()

def sum_cond(char_dict, char):
  return np.sum([char_dict[(char, i)] for i in xrange(27)])

for char in xrange(27):
  total_count = sum_cond(bigram_freq, char)
  for char2 in xrange(27):
    cond_probs[(char, char2)] = float(bigram_freq[(char, char2)])/total_count
  print char, total_count, sum_cond(cond_probs, char)


In [0]:
#fig = standard_figure()
seaborn.set_style("whitegrid")
#offsetnewbasis
fig = plt.figure(figsize=(10, 3.5))

norms = np.zeros(27)
norms_readout_space = np.zeros(27)
norms_orthogonal_space = np.zeros(27)

for i in range(27):
  #norms[i] =( np.linalg.norm(b_ixn[i]))
  norms[i] =( np.linalg.norm(b_ixn_readout[i]))
  norms_readout_space[i] = ( np.linalg.norm(b_ixn_readout[i][:27]))
  norms_orthogonal_space[i] = ( np.linalg.norm(b_ixn_readout[i][27:]))
  #print chars[ i ] , np.linalg.norm(b_h_pca[i])

#w_out_readout
#w_out_pca
#b_ixn_pca
#b_ixn_readout
  
plt.subplot(1,3,1)
ax = plt.scatter(np.log(freq)/np.log(2),norms, c = np.arange(27), cmap = 'bwr', s = 45)
plt.title("a)")
plt.xlabel("Bits of entropy")
plt.ylabel("Norm of offset vector")

for i in xrange(len(freq)):
  plt.text(np.log(freq[i])/np.log(2)+0.2, norms[i], pretty_chars[i])

plt.subplot(1,3,2)
ax = plt.scatter(np.log(freq)/np.log(2),norms_readout_space, c = np.arange(27), cmap = 'bwr', s = 45)
plt.title("b)")
plt.xlabel("Bits of entropy")
plt.ylabel("Norm of offset vector")

for i in xrange(len(freq)):
  plt.text(np.log(freq[i])/np.log(2)+0.2, norms_readout_space[i], pretty_chars[i])

plt.subplot(1,3,3)
ax = plt.scatter(np.log(freq)/np.log(2),norms_orthogonal_space, c = np.arange(27), cmap = 'bwr', s = 45)
plt.title("c)")
plt.xlabel("Bits of entropy")
plt.ylabel("Norm of offset vector")

for i in xrange(len(freq)):
  plt.text(np.log(freq[i])/np.log(2)+0.2, norms_orthogonal_space[i], pretty_chars[i])
  
#next_probs = np.array([[cond_probs[(i,j)] for j in xrange(27)] for i in xrange(27)])
# mat2 = np.array([[kl_prob(get_cond_dist(i), get_cond_dist(j)) for j in xrange(27)] for i in xrange(27)])
# mat2 = 1-np.array([[np.corrcoef(get_cond_dist(i), get_cond_dist(j))[0][1] for j in xrange(27)] for i in xrange(27)])
# ax = plt.imshow(mat2, cmap = 'gray', interpolation = 'nearest')
# plt.xticks(range(27), chars)
# plt.yticks(range(27), chars)
# plt.grid(False)
# plt.subplot(1,3,3)

# cos_dist = np.array([[sp.spatial.distance.cosine(b_ixn[i], b_ixn[j]) for j in xrange(27)] for i in xrange(27)])
# plt.imshow(-cos_dist, cmap = 'gray', interpolation = 'none')
# plt.xticks(range(27), chars)
# plt.yticks(range(27), chars)
# plt.grid(False)
plt.tight_layout()



# np.sum(mat2, axis = 0)
plt.savefig(save_dir + "bits_of_entropy.pdf")



In [0]:
#fig = standard_figure()
fig = plt.figure(figsize=(12, 4))

order = ' aeoiu'
letters_sorted = [' ','a','e','i','o', 'u']
for l in chars:
  if not l in order:
    letters_sorted.append(l)
index_func = []    
for i,_ in enumerate(letters_sorted):
  index_func.append( mapping[letters_sorted[i]])
  
    
    

components = [b_ixn_readout[index_func], b_ixn_readout[index_func,:27], b_ixn_readout[index_func,27: ] ]
titles = ['a)', 'b)', 'c)' ]
for ii in range(3):
  plt.subplot(1,3,ii+1)
  cos_dist = np.array([[sp.spatial.distance.cosine(components[ii][i], components[ii][j]) for j in xrange(27)] for i in xrange(27)])
  plt.imshow(-cos_dist, cmap = 'gray', interpolation = 'nearest')
  plt.xticks(range(27), letters_sorted)
  plt.yticks(range(27), letters_sorted)
  plt.title(titles[ii])
  plt.grid(False)

# plt.colorbar()
  
plt.savefig(save_dir + "correlation_broken_out.pdf")



In [0]:

w_x.shape
b_ixn_corrected = np.zeros_like(b_ixn)
for i in xrange(27):
  b_corrected = np.dot(w_x[i], pca.mean_) + b_ixn[i]
  b_ixn_corrected[i,:] = b_corrected
 

In [0]:
print w_out.shape
print b_ixn.shape
print out_bias.shape

chars_temp = chars[:]
chars_temp[0] = "_"

fig = standard_figure()
plt.title("Decoded bias vs actual unigram probability")
val_x =soft_max(np.reshape(out_bias, [1,-1]))

plt.subplot(1,3,1)
_ = plt.plot(val_x[0], 'r', label = "predicted")

ticks = range(0,26,5)

plt.plot(freq,'g', label = 'empirical')
#plt.title("P(char)")
plt.title("a)")
plt.xlim(0, 26)
plt.xticks(ticks, [chars_temp[tick] for tick in ticks])
plt.xlabel("Output Character")
plt.ylabel("Probability")
plt.legend()

plt.subplot(1,3,2)
#plot the conditional probs for a given char
char = 0
bias_output = np.transpose(np.dot(np.transpose(w_out), np.transpose(b_ixn))) + out_bias

bias_output_pca = np.transpose(np.dot(np.transpose(w_out), np.transpose(b_ixn_corrected))) + out_bias
plt.plot(soft_max(bias_output[char]), "r", label="predicted")
probs = [cond_probs[char, i] for i in xrange(27)]
plt.plot(probs, "g", label = "empirical")
plt.xlim(0, 26)

plt.xticks(ticks, [chars_temp[tick] for tick in ticks])
plt.xlabel("Output Character")
plt.ylabel("Conditional Probability")
#plt.title("P(char | space)")
plt.title("b)")

plt.legend(loc = 0)



#compare correlation with bigram probs vs input basis probs
corrs = []
corrs_uni = []
kl_bigrams = []
kl_unigrams = []
for char in xrange(27):
  probs_network = soft_max(bias_output[char])
  cond_p = [cond_probs[char,i] for i in xrange(27)]
  corr = np.corrcoef(probs_network, cond_p)[0][1]
  corr_unigram = np.corrcoef(cond_p, freq)[0][1]
  kl_bigram = sp.stats.entropy(cond_p, probs_network)
  kl_unigram = sp.stats.entropy(cond_p, freq)
  #print corr, corr_unigram, kl_unigram, kl_bigram
  corrs.append(corr)
  corrs_uni.append(corr_unigram)
  kl_bigrams.append(kl_bigram)
  kl_unigrams.append(kl_unigram)
  
plt.subplot(1,3,3)
ax = plt.scatter(corrs_uni, corrs, c = range(27), cmap = "hsv",  s = 45)


for k in xrange(len(kl_unigrams)):
  plt.text(corrs_uni[k]+.02, corrs[k], chars_temp[k], size=13)
plt.xlim(-.08, 1.00001)
plt.ylim(0, 1)
plt.xlabel("corr(empirical, unigram)")
plt.ylabel("corr(empirical, predicted)")
#plt.title("Predicted vs Unigram")
plt.title("c)")
x_pts = np.linspace(0, 1, 100)
print np.min(corrs_uni)
plt.plot(x_pts, x_pts, "r")
#cbar = plt.colorbar(ax)

plt.tight_layout()
# plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
plt.savefig(save_dir + "unigram_corr.pdf")

In [0]:
fig = standard_figure()

canvas1 = canvas[:, ::2]
canvas1 = canvas1.reshape((-1, 28, canvas1.shape[-1]))
final_lp = canvas1[:, :-1, -1]
canvas1 = canvas1[:,:-1,:-1]
unigram_lp = canvas1[0,:,-1]

canvas2 = np.zeros((canvas1.shape[0], canvas1.shape[-1]-1))

letters = np.zeros(canvas2.shape)

  
aggregate_low = lambda i,j: j #from step j
aggregate_high = lambda i,j: j+1 #till step j
#aggregate_high = lambda i,j: i+1 #till step i

measure = 'norm'
measure = 'xentropy'
measure = 'rev_xentropy'
measure = 'rev_kl'
measure = 'kl'

measure = 'entropy'


for i in range(canvas1.shape[0]):
  cell = canvas1[i, :, :]
  kl_divs = np.zeros((cell.shape[-1]-1, ))
  first_err = -1
  for j in xrange(kl_divs.shape[0]):
    cum_log_probs = unigram_lp + cell[:,aggregate_low(i,j):aggregate_high(i,j)].sum(1)
    letters[i, j] = np.argmax(cum_log_probs)
    if measure == "kl":
      kl_divs[j] = kl(final_lp[i, :], cum_log_probs)
    elif measure == "rev_kl":
      kl_divs[j] = kl(cum_log_probs, final_lp[i, :])
    elif measure == "norm":
      kl_divs[j] = np.linalg.norm(cum_log_probs)
    elif measure == "entropy":
      kl_divs[j] = entropy(cum_log_probs)
    elif measure == "xentropy":
      kl_divs[j] = xentropy(final_lp[i, :], cum_log_probs)
    elif measure == "rev_xentropy":
      kl_divs[j] = xentropy(cum_log_probs, final_lp[i, :])
    else:
      raise Exception("Unknown measure")
  kl_divs[i+1:] = np.nan
  canvas2[i, :] = kl_divs

plt.imshow(canvas2.T, 'Blues_r', interpolation='nearest', vmax=4)
plt.grid(False)
plt.axis('tight')
plt.xticks(*zip(*enumerate(string[1:])))
plt.yticks(*zip(*list(enumerate(string))))

if len(string)< 20:
  for i,j in zip(*np.nonzero(np.isfinite(canvas2))):
    if i==len(string)-1:
      continue
    if int(letters[i,j]) == mapping[string[i+1]]:
      color='k'
    else:
      color='r'
    plt.text(i,j+0.3,pretty_chars[int(letters[i,j])], color=color)
plt.colorbar()




In [0]:
#zoom in plot
fig = plt.figure(figsize=(14, 6))


p = plt.imshow(np.transpose(canvas[zoom_idx * 28: zoom_idx * 28 + 27, :]), 'jet', interpolation= 'nearest', vmin = -5, vmax = 10)
plt.xticks(np.arange(0, 27), chars)
label_chars = df_limit_history["input_char"][:zoom_idx+1]
for i in xrange(len(label_chars)):
  if label_chars[i] == " ":
    label_chars[i] = "''"
plt.yticks(2*np.arange(zoom_idx+1) + 0.5, label_chars)
plt.xlabel("output character")
plt.ylabel("input character")
plt.title("Breakdown of Logit Contribution")
plt.grid(False)
plt.colorbar(p)


In [0]:
plt.plot(space_norms)
plt.plot(first_letter)

In [0]:
from collections import namedtuple
Eigs = namedtuple('Eigs', ['e_vals', 'r_vecs', 'l_vecs', 'l_n_vecs'])

In [0]:
h_zero_expanded = np.hstack([h_zero, [[1]]])
w_x_expanded = np.zeros(w_x.shape + np.array([0, 1, 1]))
w_x_expanded[:,:-1,:-1] = w_x
w_x_expanded[:,:-1,-1] = b_ixn
w_o_expanded = np.zeros((27, 217))
w_o_expanded[:,:-1] = w_out.T
w_o_expanded[:,-1] = out_bias

In [0]:
eigs = []

for i, w in enumerate(w_x_expanded):
  e_vals, r_vecs = np.linalg.eig(w,)
  
  order = np.argsort(-np.abs(e_vals))
  e_vals = e_vals[order]
  r_vecs = r_vecs[:, order]
  
  l_vecs = np.linalg.pinv(r_vecs)
  
  eigs.append(Eigs(e_vals=e_vals, r_vecs=r_vecs, l_vecs=l_vecs,
                  l_n_vecs=l_vecs/np.sqrt((np.abs(l_vecs)**2).sum(1, keepdims=True))))
  
  #v0s.append(r_vecs[:1,:].T)
  label=None
  color='blue'
  if i==0:
    label='space'
    color='red'
  elif i==1:
    label='a-z'
  plt.plot(np.abs(e_vals), color=color, label=label)
  
#legend = plt.legend(loc='upper right', framealpha=1.)
#legend.get_frame().set_facecolor('#FFFFFF')
#legend.get_frame().set_alpha(1.0)
#plt.title('Singular value spectrum')
#plt.xlim(-10,225, )

In [0]:
b_ixn.shape

In [0]:
def get_word_eigs(word):
  Ww1 = np.eye(217)
  for character in word:
    Ww1 = w_x_expanded[mapping[character]].dot(Ww1)

  e_vals_w1, r_vecs_w1 = np.linalg.eig(Ww1)
  order = np.argsort(-np.abs(e_vals_w1))
  e_vals_w1 = e_vals_w1[order]
  r_vecs_w1 = r_vecs_w1[:, order]

  l_vecs_w1 = np.linalg.pinv(r_vecs_w1)
  return Ww1, Eigs(e_vals=e_vals_w1, r_vecs=r_vecs_w1, l_vecs=l_vecs_w1,
                   l_n_vecs=l_vecs_w1/np.sqrt((np.abs(l_vecs_w1)**2
                                              ).sum(1, keepdims=True)))

In [0]:
plt.figure(figsize=(15,15), dpi=150)
#h0 = np.vstack((b_ixn[mapping[' ']][:,None], [[1]]))

Ww1, eigs_w1 = get_word_eigs(' united')
Ww2, eigs_w2 = get_word_eigs('states')

eigs_s = eigs[mapping[' ']]
lambda_s_sq = np.diag(np.sqrt(eigs_s.e_vals))

LSR1 = lambda_s_sq.dot(eigs_s.l_vecs).dot(eigs_w1.r_vecs)
L2RS = eigs_w2.l_vecs.dot(eigs_s.r_vecs.dot(lambda_s_sq))

LSR1_power = (np.abs(LSR1)**2).sum(1)
L2RS_power = (np.abs(L2RS)**2).sum(0)

plt.subplot(2,2,1)
plt.imshow(np.abs(LSR1), 'jet', interpolation='nearest')
plt.colorbar()

plt.subplot(2,2,2)
plt.imshow(np.abs(L2RS), 'jet', interpolation='nearest')
plt.colorbar()

plt.subplot(2,2,3)
img_overlap = np.zeros((217,217,3))
img_overlap[:,:,0] = np.abs(LSR1) / np.max(np.abs(LSR1))
img_overlap[:,:,1] = np.abs(L2RS) / np.max(np.abs(L2RS))
plt.imshow(img_overlap, interpolation='nearest')

#plt.xlim(0,25)
#plt.ylim(25,0)

plt.subplot(2,2,4)
plt.semilogy(LSR1_power, label='LSR1_power')
plt.semilogy(L2RS_power, label='L2RS_power')
plt.legend()

np.corrcoef(LSR1_power, L2RS_power)

In [0]:
def pseudso_ent(x):
  x = np.abs(x)
  x /= x.sum()
  return -(np.log(x + 1e-20) * x).sum()

In [0]:
keep_dim = 50
res = np.zeros((27, 27))
for i in xrange(27):
  for j in xrange(27):
    MM = eigs[j].l_n_vecs.dot(eigs[i].r_vecs)
    res[i, j] = pseudo_ent(MM[:keep_dim, :keep_dim])
    # plt.imshow(np.abs(MM),'jet')
    # plt.colorbar()

plt.imshow(res)

In [0]:
res[np.arange(27),np.arange(27)] = np.nan
plt.imshow(res, 'jet', interpolation='nearest')
plt.colorbar()
plt.xticks(*zip(*enumerate(pretty_chars)))
plt.yticks(*zip(*enumerate(pretty_chars)))
None

In [0]:
 np.diag(np.sqrt(eigs[mapping[bigram[1]]].e_vals)
             ).shape

In [0]:
bigram='th'
sL = np.diag(np.sqrt(eigs[mapping[bigram[1]]].e_vals)
             ).dot(eigs[mapping[bigram[1]]].l_vecs)
sR = eigs[mapping[bigram[0]]].r_vecs.dot(
    np.diag(np.sqrt(eigs[mapping[bigram[0]]].e_vals)))

MM = sL.dot(sR)
plt.imshow(np.abs(MM),'jet', interpolation='nearest')
plt.colorbar()
#plt.xlim(0,15)
#plt.ylim(0,15)

plt.figure()
plt.hist(np.abs(MM.ravel()), 256)
None

In [0]:
bigram='th'
sL = np.diag(np.sqrt(eigs[mapping[bigram[1]]].e_vals)
             ).dot(eigs[mapping[bigram[1]]].l_vecs)
sR = eigs[mapping[bigram[0]]].r_vecs.dot(
    np.diag(np.sqrt(eigs[mapping[bigram[0]]].e_vals)))

MM = sL.dot(sR)
plt.imshow(np.abs(MM),'jet', interpolation='nearest')
plt.colorbar()
#plt.xlim(0,15)
#plt.ylim(0,15)

plt.figure()
plt.hist(np.abs(MM.ravel()), 256)
None

In [0]:
MM.dot(MM1)

In [0]:
plt.scatter(np.real(e_vals), np.imag(e_vals))
plt.axis('square')

In [0]:
np.max(np.abs(l_vecs.dot(w) - np.diag(e_vals).dot(l_vecs)))

In [0]:
train_string = ' annual revenue increased beyond expectations '
hs = [h_zero.T]

for char in train_string:
  hs.append(make_step(hs[-1], mapping[char])[0])

hs = np.hstack(hs[1:])

space_logs = (w_out.T.dot(hs) + out_bias[:,None])[mapping[' ']]

In [0]:
plt.plot(space_logs)
plt.plot(np.real(r_vecs[:, :1]).T.dot(hs).T)
plt.xticks(*zip(*enumerate(train_string)))
None

In [0]:
#plt.plot(np.real(l_vecs[:1,:]).dot(hs).ravel())

plt.scatter(np.real(l_vecs[:1,:]).dot(hs).ravel(), space_logs, alpha=0.1, marker='.')

In [0]:
b_ixn.shape

In [0]:
probs = [cond_probs[(mapping['q'],i)] for i in xrange(27)]
plt.plot(probs )
plt.xticks(range(27), pretty_chars)
None
print entropy_probs(probs)

In [0]:
print np.dot(w_out.T, pca.mean_).shape
np.linalg.norm(np.dot(w_out.T, pca.mean_))/np.linalg.norm(pca.mean_)

In [0]:
plt.plot(pca.mean_)

In [0]:
bias_output_pca[0]

In [0]:
bias_output[0]

In [0]:
char = 5
#bias_output_pca = np.transpose(np.dot(np.transpose(w_out), np.transpose(b_ixn_corrected))) + out_bias
plt.plot(soft_max(bias_output[char]), "b", label="predicted")
plt.plot(soft_max(bias_output_pca[char]), "r", label="predicted_pca")
probs = [cond_probs[char, i] for i in xrange(27)]
plt.plot(probs, "g", label = "empirical")

In [0]:
pca.mean_.shape

In [0]:
print i 
print j 
print df_limit_history["input_history"][0]

In [0]:
x_lim = 50
#normalising = np.sum(( logit_val[:,:x_lim+1,i]  != 0), 0 )
y_s =  (logit_val[:,:,i].sum(0)[:x_lim+1]) #/normalising
x_s =  (vals[:,:,i].sum(0)[:x_lim+1]) #/normalising

plt.plot(y_s / x_s)


In [0]:
#fig2 NORM OF ELEMENTS IN THE BUFFER
errors = np.zeros(50)
#h  = np.zeros((1, num_nodes))
h  = h_zero
losses = []
counter = 0
max_history = 200
history = []
input_history = [0 for i in xrange(max_history)]
#df_limit_history = pd.DataFrame()

count = 0
for i in range(0,60000):
  if i % 1000 == 0:
    print i, len(df_limit_history)
  h_in = h
  input_val = np.argmax(x[i])
  for k in range(min(max_history, count)):
    new_h = np.dot( w_x[input_val], np.reshape( history[k], [-1,1] ))
    out_val = out_val + new_h
    history[k] = new_h
    
  out_val = np.zeros((num_nodes,1))
  for k in range(min(max_history, count)):
    out_val = out_val +  history[k]
    
  h = out_val +  np.reshape(b_ixn[input_val], [-1,1] ) 
  if len( history) == max_history: 
    history[-1] = history[-1] + history[-2]
    history.pop(-2)
    input_history.pop(-2)
  history.insert(0,np.reshape(b_ixn[input_val], [-1,1] ) )
  input_history.insert(0,input_val)
  outpt_np = np.dot( np.reshape(h, [1,-1]), w_out) + out_bias
  
  #outpt_np = np.dot( np.reshape(h, [1,-1]), w_out) + out_bias
  loss = sess.run( [train_loss_np], feed_dict={output_placeholder: outpt_np, target_placeholder:y[i]})
  outpt_np_new = outpt_np
  for t in range(len(history)-1, -1,-1):
    outpt_np = outpt_np - np.dot( np.reshape(history[t], [1,-1]), w_out)[0]  
    if np.argmax( outpt_np_new ) != np.argmax(outpt_np):
      #print t, input_history[t]
      d = {"index" : i, "input":np.argmax(x[i]), "distance": t, "trigger_offset": input_history[t]}
      df_limit_history = df_limit_history.append(d, ignore_index=True)
      break
    if t == 0:
      d = {"index" : i, "input":np.argmax(x[i]), "distance": t, "trigger_offset": input_history[t]}
      df_limit_history = df_limit_history.append(d, ignore_index=True)
  count += 1
  if i  > 10:
    losses.append(loss)
#     d = {"index" : i, "state": h, "input":np.argmax(x[i]), "state_in":h_in, "delta":h-h_in, "dist": i % 50, "loss": loss, "history":history[:], "input_history": input_history[:] }
#     df_limit_history = df_limit_history.append(d, ignore_index=True)
print("produced {} random states".format(n_random))
print(np.mean(losses))

In [0]:
fig = standard_figure()

# plt.subplot(1,2,1)
# _= plt.hist(df_limit_history["distance"].values,bins= 100)

mean_by_char = df_limit_history.groupby(['trigger_offset',"input"])["distance"].mean()
count = df_limit_history.groupby(['trigger_offset',"input"])["distance"].count()
# without_space = df_limit_history_space.groupby(['counter'])["loss_other"].median().values
# all_chars = df_limit_history_space.groupby(['counter'])["loss"].median().values

canvas = np.zeros((27,27))*np.nan
canvas_count = np.zeros((27,27))*np.nan

for i in range(27):
  for j in range(27):
    temp = df_limit_history[ (df_limit_history["trigger_offset"] == j) & (df_limit_history["input"] == i) & (df_limit_history["distance"] >0 )]
    canvas_count[i,j] = len(temp["distance"]) 

    if len(temp["distance"]) > 10:
      canvas[i,j] = (temp["distance"].median())
ax1 = plt.subplot(1,2,1)
ax = plt.imshow(canvas, cmap = 'jet', interpolation = 'nearest')
plt.xticks(*zip(*enumerate(pretty_chars[:])))
plt.yticks(*zip(*list(enumerate(pretty_chars))))
ax1.yaxis.grid(False)
ax1.xaxis.grid(False)
fig.colorbar(ax)

ax1 = plt.subplot(1,2,2)
ax = plt.imshow(canvas_count, cmap = 'jet', interpolation = 'nearest')
plt.xticks(*zip(*enumerate(pretty_chars[:])))
plt.yticks(*zip(*list(enumerate(pretty_chars))))
ax1.yaxis.grid(False)
ax1.xaxis.grid(False)

fig.colorbar(ax)





In [0]:

plt.plot(100*pca.explained_variance_ratio_, 'b')
plt.title("Explained Variance Ratio")
plt.xlabel("Number of PCA dimension")
plt.ylabel("%")

In [0]:
norms = np.zeros(27)
for i in range(0,27):
  norms[i] = (np.linalg.norm(b_ixn[i]))
  
print norms.mean()

In [0]:
np.linalg.norm(b_ixn[0])

In [0]:
#  #soft_max(sum_of_hm1_cont_super + out_bias).shape
# plt.imshow(np.transpose(superprobcanvas))



In [0]:
fig = standard_figure()

canvas1 = canvas[:, ::2]
canvas1 = canvas1.reshape((-1, 28, canvas1.shape[-1]))
final_lp = canvas1[:, :-1, -1]
canvas1 = canvas1[:,:-1,:-1]
unigram_lp = canvas1[0,:,-1]

canvas2 = np.zeros((canvas1.shape[0], canvas1.shape[-1]-1))

letters = np.zeros(canvas2.shape)

  
aggregate_low = lambda i,j: j #from step j
aggregate_high = lambda i,j: j+1 #till step j
#aggregate_high = lambda i,j: i+1 #till step i

measure = 'norm'

measure = 'xentropy'
measure = 'rev_xentropy'
measure = 'rev_kl'
measure = 'kl'

measure = 'norm'
measure = 'entropy'

spc_count = 0

for i in range(canvas1.shape[0]):
  spc_count += string[i] == ' '
  cell = canvas1[i, :, :]
  kl_divs = np.zeros((cell.shape[-1]-1, ))
  first_err = -1
  for j in xrange(kl_divs.shape[0]):
    cum_log_probs = unigram_lp + cell[:,aggregate_low(i,j):aggregate_high(i,j)].sum(1)
    letters[i, j] = np.argmax(cum_log_probs)
    if measure == "kl":
      kl_divs[j] = kl(final_lp[i, :], cum_log_probs)
    elif measure == "rev_kl":
      kl_divs[j] = kl(cum_log_probs, final_lp[i, :])
    elif measure == "norm":
      kl_divs[j] = np.linalg.norm(cum_log_probs)
    elif measure == "entropy":
      kl_divs[j] = entropy(cum_log_probs)
    elif measure == "xentropy":
      kl_divs[j] = xentropy(final_lp[i, :], cum_log_probs)
    elif measure == "rev_xentropy":
      kl_divs[j] = xentropy(cum_log_probs, final_lp[i, :])
    else:
      raise Exception("Unknown measure")
  kl_divs[spc_count:] = np.nan
  canvas2[i, :] = kl_divs

plt.imshow(canvas2.T, 'Blues_r', interpolation='nearest')
plt.grid(False)
plt.axis('tight')
plt.xticks(*zip(*enumerate(string[1:])))
plt.yticks(*zip(*list(enumerate(string.strip().split()))))

spcs = np.zeros((len(string),), dtype="|S1")
spcs[:] = list(string)
spcs = np.nonzero(spcs==' ')[0]
spcs = np.stack([spcs-1.5, spcs-0.5]).reshape(-1)

import matplotlib.ticker
plt.gca().xaxis.set_minor_locator(matplotlib.ticker.FixedLocator(spcs))
plt.gca().yaxis.set_minor_locator(matplotlib.ticker.FixedLocator(np.arange(canvas2.shape[1])-0.5))

plt.grid(which='minor')

if len(string)< 20:
  for i,j in zip(*np.nonzero(np.isfinite(canvas2))):
    if i==len(string)-1:
      continue
    if int(letters[i,j]) == mapping[string[i+1]]:
      color='k'
    else:
      color='r'
    plt.text(i,j+0.3,pretty_chars[int(letters[i,j])], color=color)
plt.colorbar()

In [0]:
superprobcanvas[i, k ] = 
np.reshape( soft_max(outpt_np) ,[-1,1])[ mapping[string[i+1]]]


In [0]:
def predict(input_string, n, t, h = None, on_input = False, random_inputs = False, transform = None):
  indx = range(len(x)-(n + len(input_string)))
  np.random.shuffle(indx)
  string = input_string
  values = np.zeros((len(input_string) + n, num_nodes))
  all_inputs = np.zeros((len(input_string) + n, num_nodes))
  surprise = np.zeros((len(input_string) + n+1))
  certainty = np.zeros((len(input_string)+ n))
  input_vals = np.zeros((len(input_string) + n))
  if h is None:
    h  = h_zero
  else:
    h = np.reshape(h, [1,-1])
  counter = 0
  for i in range(len(input_string) + n):
    h_in = h
    if on_input:
      if random_inputs or random_inputs:
          inputs_t = x[indx[i]]
      else:
          inputs_t = x[i+indx[0]]
      string += chars[np.argmax(inputs_t)]
    else:
      inputs_t = np.zeros((1,27))
      if i < len(input_string):
        inputs_t[0, mapping[ input_string[i]]] = 1
      else:
        inputs_t[0,choice] = 1
        string = string + (char_out)
    h, output_now = make_step(h, np.argmax(inputs_t))

    #h,output_now, probs = sess.run( [hidden_state_source_pca, output_source_pca, output_eval_pca], feed_dict={temperature:[t]})
    probs  = soft_max(output_now, t)
    choice = np.random.choice(len(probs[0,:]), p=probs[0])
    char_out = chars[choice] 
    values[i] = np.transpose(h)
    all_inputs[i] = b_ixn[np.argmax(inputs_t)]
    input_vals[i] = np.argmax(inputs_t)
    certainty[i] = np.sum( probs[0,:]*np.log(probs[0,:])/np.log(2))
    if on_input:
      surprise[i+1] = np.log(probs[0,np.argmax(y[i])])/np.log(2)
    else:
      if i < len(input_string)-1: 
        surprise[i+1] = np.log(probs[0,mapping[input_string[i+1]]])/np.log(2)
      else:
        surprise[i+1] = np.log(probs[0,choice])/np.log(2)
  if not (transform is None ):
    values = np.transpose( transform.dot(np.transpose(values)))
    all_inputs = np.transpose(transform.dot(np.transpose(all_inputs)))
    print "did transform"

  return values, output_now, string, all_inputs, -surprise, input_vals, certainty

In [0]:
values, output_now, string, all_inputs, surprise, input_vals, certainty = predict(" a lot ", 100, 2)

values, output_now, string_annual, all_inputs, surprise, input_vals, certainty = predict(" show me ing", 100, 20)
print string
print string_annual

In [0]:
1+1

In [0]:
for word in words:
  word_counts[word]+=1

In [0]:
word_counts.most_common()[:20]

In [0]:
def get_word_vec(word):
  vec = b_ixn[0]
  for c in word:
    vec = np.dot(w_x[mapping[c]], vec)
  return vec

def get_space_norm(word, char = ' '):
  vec = get_word_vec(word)
  image = np.dot(w_x[mapping[char]], vec)
  
  return np.linalg.norm(image)/np.linalg.norm(vec)

word_tups = []
summary_df = pd.DataFrame()
for word, count in word_counts.most_common():
  if i %10000 == 0:
    print i, len(summary_df)

  d = {"word":word, "shrinkage":get_space_norm(word, ' '), "count":count}
  summary_df = summary_df.append(d, ignore_index=True)

print len(summary_df)

summary_df = summary_df.drop_duplicates()
print len(summary_df)


In [0]:
plt.plot(summary_df["count"], summary_df["shrinkage"], 'r.')
plt.xlim(0, 2000)

In [0]:
word_tups_sorted[-40:]

In [0]:
word_tups_sorted[:40]

In [0]:
all_state, _, next_val, input_val, suprise, input_indx, certainty = predict("this is the life of someone who is living on the edg", 50, 10, on_input = False, random_inputs = False) #basis
fig = plt.figure(figsize=(28, 12))
print(next_val)
plt.subplot(2,4,1)
_ = plt.plot(all_state[:,:50])
plt.title("first 50 dimensions plotted as function of time")
plt.subplot(2,4,2)
_ = plt.plot(np.transpose(all_state))
plt.title("all dimensions plotted each line is one time step")

ax1 = plt.subplot(2,4,3)
ax1.plot((np.sum(all_state**2,1)**0.5))
plt.title("blue: norm of activation, red: certainty")

suprise = suprise
ax2 = ax1.twinx()

#ax2.plot(suprise, 'r')
ax2.plot(certainty, 'r')

plt.subplot(2,4,4)
_ = plt.plot(np.log((np.mean(all_state**2,0))))
plt.title("log of squared activations by dimension")


plt.subplot(2,4,5)
plt.title("activations over time")

_  = plt.imshow(np.transpose(all_state[:,:100]),interpolation = 'none',cmap ='bwr')

plt.subplot(2,4,6)
plt.title("input offsets over time")

ax = plt.imshow(np.transpose(input_val[:,:100]),interpolation = 'none',cmap ='bwr')

print(np.corrcoef(suprise[:-1], (np.sum(all_state**2,1)**0.5)))
print(np.corrcoef(certainty, (np.sum(all_state**2,1)**0.5)))




plt.subplot(2,4,7)
plt.title("state - input over time")

plt.title("")

ax = plt.imshow(np.transpose(all_state[:,:100]-input_val[:,:100]),interpolation = 'none',cmap ='bwr')

#ax = plt.scatter(suprise[:-1],  np.sum(all_state[:,:]**2,1), c = input_indx, cmap ='bwr')
ax1 = plt.subplot(2,4,8)
delta_norm = (np.sum((all_state[0:-1] -all_state[1:])**2,1))
ax1.plot(delta_norm)

print(np.corrcoef(suprise[:-2], delta_norm))


ax2 = ax1.twinx()

ax2.plot(suprise, 'r')
plt.title("blue: change of activation, red: is surprise")

#ax2.plot(certainty, 'r')

plt.savefig("/tmp/comprehensive_view.pdf")

In [0]:
#power_plot

losses = []
count = 0
h  = h_zero
counter = 0
h_space = h
h_other = h
random_states = []
losses = []
split = np.zeros((5000,2))
inputs = []
inputs_num = []
counter = 0
df_temp = pd.DataFrame()
for i in range(5000):
  h_in = h
  input_val = np.argmax(x[i])
  if input_val ==0:
    counter = 0
  else:
    counter = counter + 1
  h, outpt_np, loss =  make_step(h, input_val, y[i])
  h_basis_new = np.transpose(np.dot(b_1,  (h)))
  split[i,0] = np.linalg.norm( h_basis_new )**2
  split[i,1]= np.linalg.norm( h_basis_new[0][:27])**2
  #split[i,1]= np.linalg.norm( h_basis_new[0][27:])/(216-27)
  inputs.append(pretty_chars[input_val])
  inputs_num.append(counter)
  d = {"input_val":counter, "length_one":np.linalg.norm( h_basis_new )**2 ,  "length_one":np.linalg.norm( h_basis_new )**2


  
 

In [0]:
fig = plt.figure(figsize=(10,10))

# plt.plot(split[:,1])
# plt.plot(split[:,0])
# plt.xticks(*zip(*list(enumerate(inputs))))
# None

# # #abs((b_1.T.dot(b_1)-np.eye(216))).sum()
# # # if split[:,0] > 0:
# # #   x_ax = split[:,1]/split[:,0]
# # # else:
# # #   x_ax 
# # plt.subplot(1,2,2)
# # plt.scatter(x_ax[1:], np.array( inputs_num[0:-1]))
# # #readout_basis
# # #print x_ax[:5]
# # #print inputs_num[:5]

# #plt.plot(x_ax[:50],inputs_num[:50], '.')
#plt.scatter(np.ones(10), np.zeros(10))
plt.scatter(np.array( inputs_num[0:]), x_ax[:], )


In [0]:

#Figure:BasisTransform pca_and_readout
fig = standard_figure(5)

matrices = [w_out, b_ixn, w_out_pca, b_ixn_pca, w_out_readout, b_ixn_readout] 

plot_pre_fix = ["Readout ", "Biases "]
basis_label = ["Original", "PCA" , "Readout" ]
basises = [np.eye(num_nodes), pca_basis, readout_basis]


counter = 0
#import matplotlib.gridspec as gridspec
#gs = gridspec.GridSpec(2, 2)
#ax = plt.subplot(gs[0, 0])

for row in range(3):
  for col in range(2):
    if col == 0:
      matrix = np.transpose(matrices[counter])
    else:
      matrix = matrices[counter]
      
    title = plot_pre_fix[col] + " Matrix in "+ basis_label[row] + " Basis"
    
    #ax = plt.subplot2grid((3,7), (row, col*3), colspan=3) #plt.subplot(3,2,counter + 1)
    ax = plt.subplot(3,2,counter + 1)
    plt.title(title)
    plt.imshow(matrix,interpolation = 'nearest',cmap ='bwr', vmin = -2, vmax = 2)
    counter = counter + 1
    plt.yticks(*zip(*list(enumerate(pretty_chars))[::6]))
    plt.xlabel("dimension")

# import matplotlib as mpl
# plt.subplot2grid((3,7), (1,6))
# #mpl.colorbar.ColorbarBase(ax)
# plt.imshow(np.zeros((0,0)), vmin=-2, vmax=2)
# _= fig.colorbar(ax, ticks=[])




In [0]:
fig = plt.figure(figsize=(10,20))
 
w_inn = np.transpose(b_ixn)
w_out_temp = np.transpose(w_out)
print w_out_temp.shape
print w_inn.shape

u,s,v = np.linalg.svd(w_out_temp)
temp = w_out_temp.dot(np.transpose(v))
b_1_inv = np.transpose(v)
b_1 = np.linalg.inv(b_1_inv)
q_1 = w_out_temp.dot(b_1_inv)[:27,:27]

block_1 = np.identity(num_nodes)
for i in range(27):
  q_1[i] = q_1[i] / np.linalg.norm(w_out_temp[i])
block_1[:27,:27] = q_1



plt.subplot(6,1,1)
plt.imshow(w_out_temp.dot(b_1_inv))

plt.subplot(6,1,2)
basis_1 = block_1.dot(b_1)
plt.imshow(w_out_temp.dot(np.linalg.inv(basis_1)))

plt.subplot(6,1,3)
w_til_in = basis_1.dot(w_inn)
w_til_in_sub = w_til_in[27:,:]
u,s,v = np.linalg.svd(w_til_in_sub)

b_2 = np.transpose(u)
plt.imshow(np.transpose(b_2.dot(w_til_in_sub)))

block_b2 = np.identity(num_nodes)
block_b2[27:,27:]=b_2
q_2 = b_2.dot(w_til_in_sub)[:27,:27]

# for i in range(27):
#   q_2[i] = q_2[i] / (np.linalg.norm(w_inn[:,i])

block_2 = np.identity(num_nodes)
block_2[27:2*27,27:2*27] = q_2

plt.subplot(6,1,4)
plt.imshow(np.transpose(np.linalg.inv(block_2).dot(block_b2).dot(w_til_in)))

#basis_complete maps h to h_tilde, basis_complete.dot(h) = h_tilde
basis_complete = np.linalg.inv(block_2).dot(block_b2).dot(basis_1)

plt.subplot(6,1,5)
plt.imshow(np.transpose(basis_complete.dot(w_inn)),'bwr')


plt.subplot(6,1,6)
plt.imshow(w_out_temp.dot(np.linalg.inv(basis_complete)),'bwr')

random_states_pca_transformed = np.zeros((len(random_states), num_nodes))
for i in range(len(random_states)):
  random_states_pca_transformed[i] = (basis_complete).dot(np.transpose(random_states[i]))

pca_2 = PCA(n_components=pca_dim - 54)
pca_2.fit(random_states_pca_transformed[:][54:])
#pca_states_transform = random_states_pca[:,54:]
#pca_states_transform.var(1).mean()
#pca_states_transformed = (basis_complete).dot(np.transpose(random_states_pca))
block_3 = np.identity(num_nodes)
block_3[54:,54:] = (pca_2.components_)
basis_complete = block_3.dot(np.linalg.inv(block_2)).dot(block_b2).dot(basis_1)


In [0]:
plt.imshow(w_out.T.dot(b_1.T))




In [0]:
fig = plt.figure(figsize=(8, 8))
ax = scatter_plot(df_states["state"], df_states["input"])
cbar = fig.colorbar(ax)

plt.savefig("/tmp/pca_plot_hidden_state_by_input.pdf") 
  
  #PCA PLOT


# np.dot(pca.components_, np.transpose(pca.components_))


# w_h_inn = sess.run(source_rnn_cell._Wh_ixnn)
# b_ixn = sess.run(source_rnn_cell._b_ixn)
# out_inn = sess.run( source_rnn_cell._Wro_nxo)
# init =  sess.run( source_rnn_cell._init_vector)

# w_h_pca = np.zeros((27, pca_dim**2))
# b_h_pca = np.zeros((27, pca_dim))
# out_pca = np.dot( (pca.components_), out_inn)
# init_pca = np.dot( (pca.components_), np.transpose(init))
# w_h_pca_nxn = np.zeros((27, pca_dim, pca_dim ))
# w_h_nxn =  np.zeros((27, num_nodes, num_nodes ))
# for i in range(27):
#   tmp = np.reshape(w_h_inn[i, :], [num_nodes, num_nodes] )
#   w_h_nxn[i] = tmp
#   tmp_2 = np.dot(pca.components_, np.dot(tmp, np.transpose(pca.components_)))
#   w_h_pca[i,:] = np.reshape( tmp_2, [-1]) 
#   w_h_pca_nxn[i] = tmp_2
#   b_h_pca[i,:] =  np.dot(b_ixn[i,:], np.transpose(pca.components_))

# source_rnn_cell_pca = source_cell_class(input_size, pca_dim, output_size, hparams, layer = 'only')

# _ = sess.run(source_rnn_cell_pca._Wh_ixnn.assign(w_h_pca))
# _ = sess.run(source_rnn_cell_pca._b_ixn.assign(b_h_pca))
# _ = sess.run( source_rnn_cell_pca._Wro_nxo.assign(out_pca))
# out_original_bias = sess.run( source_rnn_cell._bro_o )
# _ = sess.run( source_rnn_cell_pca._bro_o.assign(out_original_bias))

# _ =  sess.run( source_rnn_cell_pca._init_vector.assign(np.transpose(init_pca)))


# _ = sess.run(source_rnn_cell_pca._x0_ixn.assign(np.zeros((10,27,pca_dim))))


# state_placeholder_pca = tf.placeholder(tf.float32, shape=[None, pca_dim])
# output_source_pca, hidden_state_source_pca = source_rnn_cell_pca(input_placeholder, state_placeholder_pca)



# output_eval_pca = tf.nn.softmax( output_source_pca * temperature)

# output_placeholder = tf.placeholder(tf.float32, shape=[None, 27])

# softmax = tf.nn.softmax( output_placeholder * temperature)
# output_eval_pca = tf.nn.softmax( output_source_pca * temperature)

# train_loss_np  = tf.reduce_mean(
#       tf.nn.softmax_cross_entropy_with_logits(
#         output_placeholder, target_placeholder) / tf.log(2.0) )

# train_loss_pca  = tf.reduce_mean(
#       tf.nn.softmax_cross_entropy_with_logits(
#         output_source_pca, target_placeholder) / tf.log(2.0) )

In [0]:
fig = plt.figure(figsize=(20, 15))
losses = []
count = 0
h  = h_zero
counter = 0
h_space = h
h_other = h
input_vals = " sample"
for i in range(len(input_vals)):
  h_in = h
  input_val = mapping[input_vals[i]]

  h_space, outpt_np_space, loss_space = make_step(h_space, input_val, y[i])
  h_other, outpt_np_other, loss_other = make_step(h_other, input_val, y[i])
  h, outpt_np, loss =  make_step(h, input_val, y[i])
  input_vals += chars[input_val]

print(input_vals)
backwards_val = ""
for i in range(len(input_vals)):
  probs = np.zeros(27)
  for k in range(27):
    _, p =make_step_back(h,k)
    probs[k] = p
  print(probs)
  input_val = np.argmax(probs)
  h, _ = make_step_back(h,input_val)
  backwards_val += chars[input_val]
print(backwards_val)
  
    

In [0]:
pca.explained_variance_ratio_[-5:]
cond_probs

In [0]:
max_history = 5
n_random = 20000

#define the inputs
index = range(n_random) 
np.random.shuffle(index)
inputs = np.zeros((27*num_each_char,27))
for i in range(27*num_each_char):
  inputs[i, i % 27] = 1

# This produces n_random hidden states by running the RNN in feed forward mode with the input. We could add a random sampling probability here.
df_states = pd.DataFrame()
errors = np.zeros(50)
#h  = np.zeros((1, num_nodes))
h  = sess.run(source_rnn_cell.zero_state(1, dtype=tf.float32))
losses = []

history = [0 for i in xrange(max_history)]
for i in range(n_random):
  if i % 1000 == 0:
    print i, len(df_states)
  if i % 50 == 0:
    h  = sess.run(source_rnn_cell.zero_state(1, dtype=tf.float32))
  h_in = h
  h,loss = sess.run( [hidden_state_source, train_loss], feed_dict={input_placeholder:x[i], state_placeholder:h, target_placeholder:y[i]})
  history.pop(0)
  history.append(np.argmax(x[i]))

  if i % 50 > 10:
    losses.append(loss)
    d = {"index" : i, "state": h, "input":np.argmax(x[i]), "state_in":h_in, "delta":h-h_in, "dist": i % 50, "loss": loss, "history":history[:] }
    df_states = df_states.append(d, ignore_index=True)
print("produced {} random states".format(n_random))
print(np.mean(losses))
print len(df_states)

random_states = np.array([v[0] for v in df_states["state"].values])

In [0]:
#history from nearest neighbor
idx = 580
s = df_states["state"].values[idx]
def get_dist(s1, s2):
  d = (s1 - s2)**2
  return np.sum(d)

dists = [get_dist(s, s1) for s1 in df_states["state"]]
sorted_args = np.argsort(dists)
print dists[sorted_args[0]], dists[sorted_args[1]]
print df_states["history"].values[idx]

for i in xrange(10):
  print dists[sorted_args[i]], get_chars(df_states["history"].values[sorted_args[i]])
  
#Can we plot this nicely? Distance in space as a function of difference in characters? Also, I think we should be thinking about angular distance. Eg. cosine distance.

In [0]:
fig = plt.figure(figsize=(15, 7))

mapping = {}
for i in range(27):
  mapping[chars[i]] = i
  

freq = np.zeros(27)
for i in x:
  freq[np.argmax(i)] +=1
  
freq = freq / np.sum(freq)
plt.plot(freq,'r')
                           

outpt_np = np.dot( b_h_pca[:,:],out_pca) + out_original_bias

temp = b_h_pca
temp[:,30:] = 0
outpt_corrupt = np.dot( temp,out_pca) + out_original_bias

plt.subplot(1,2,1)
plt.title("Decoded prio vs actual")
val_x =soft_max(np.reshape(out_original_bias, [1,-1]))
print val_x.shape
print(np.sum(val_x, 1))
_ = plt.plot(val_x[0])

plt.plot(freq,'r')
plt.subplot(1,2,2)
plt.title("Decoded probabilities from offset")

for i in range(27):
  _ =plt.plot(soft_max(outpt_np[i]))
  print 'char ', chars[i], 'next ', chars[np.argmax(outpt_np[i])], 'corrupted ', chars[np.argmax(outpt_corrupt[i])]

plt.savefig("/tmp/prior_vs_actual.pdf")
 #Case study: "United States" vs "United Nations"


In [0]:
#random_states_pca.shape

In [0]:
#Sample PCA states and compute PCA error
max_history = 5
n_random = 10000

#define the inputs
index = range(n_random) 
np.random.shuffle(index)
inputs = np.zeros((27*num_each_char,27))
for i in range(27*num_each_char):
  inputs[i, i % 27] = 1

#Hack: Overwrite all of the offsets with the same value (since they are correlated)
if False:
  temp = b_h_pca
  for i in range(27):
    temp[i] = np.mean(b_h_pca,0) 
  _ = sess.run(source_rnn_cell_pca._b_ixn.assign(temp))
else:
  _ = sess.run(source_rnn_cell_pca._b_ixn.assign(b_h_pca))
  
  
# This produces n_random hidden states by running the RNN in feed forward mode with the input. We could add a random sampling probability here.
df_states_pca = pd.DataFrame()
errors = np.zeros(50)
#h  = np.zeros((1, num_nodes))
h  = sess.run(source_rnn_cell_pca.zero_state(1, dtype=tf.float32))
losses = []
counter = 0
for i in range(0,n_random):
  if i % 1000 == 0:
    print i, len(df_states)
  if i % 50 == 0:
    h  = sess.run(source_rnn_cell_pca.zero_state(1, dtype=tf.float32))
  h_in = h
  h,loss = sess.run( [hidden_state_source_pca, train_loss_pca], feed_dict={input_placeholder:x[i], state_placeholder_pca:h, target_placeholder:y[i]})

  if i % 50 > 10:
    losses.append(loss)
    d = {"index" : i, "state": h, "input":np.argmax(x[i]), "state_in":h_in, "delta":h-h_in, "dist": i % 50, "loss": loss, "history":history[:] }
    df_states_pca = df_states_pca.append(d, ignore_index=True)
print("produced {} random states".format(n_random))
print(np.mean(losses))
print len(df_states_pca)
random_states_pca = np.array([v[0] for v in df_states_pca["state"].values])



In [0]:
#fig2
#Barebones running of the RNN with all elements accessible in numpy. Also explores a limited buffer of linearly independent propagated offsets
errors = np.zeros(50)
#h  = np.zeros((1, num_nodes))
h  = sess.run(source_rnn_cell_pca.zero_state(1, dtype=tf.float32))
losses = []
counter = 0
max_history = 200
history = [0 for i in xrange(max_history)]

df_limit_history = pd.DataFrame()

count = 0
for i in range(10000,15000):
  if i % 1000 == 0:
    print i, len(df_states)
#   if i % 50 == 0:
#     h  = sess.run(source_rnn_cell_pca.zero_state(1, dtype=tf.float32))
  h_in = h
  input_val = np.argmax(x[i])
  for k in range(min(max_history, count)):
    new_h = np.dot( w_h_pca_nxn[input_val], np.reshape( history[k], [-1,1] ))
    out_val = out_val + new_h
    history[k] = new_h
    
  out_val = np.zeros((num_nodes,1))
  for k in range(min(max_history, count)):
    if not k > max_history:
      out_val = out_val +  history[k]
  h = out_val +  np.reshape(b_h_pca[input_val], [-1,1] ) 
  if len( history) == max_history: 
    history[-1] = history[-1] + history[-2]
    history.pop(-2)
  history.insert(0,np.reshape(b_h_pca[input_val], [-1,1] ) )
  
  outpt_np = np.dot( np.reshape(h, [1,-1]), out_pca) + out_original_bias
  loss = sess.run( [train_loss_np], feed_dict={output_placeholder: outpt_np, target_placeholder:y[i]})
  count += 1
  if i  > 10:
    losses.append(loss)
    d = {"index" : i, "state": h, "input":np.argmax(x[i]), "state_in":h_in, "delta":h-h_in, "dist": i % 50, "loss": loss, "history":history[:] }
    df_limit_history = df_limit_history.append(d, ignore_index=True)
print("produced {} random states".format(n_random))
print(np.mean(losses))


#droppping the most recent offset from the current prediction reduces accuracy to 1.9, droppping the 1st contribution to 1.718, 2nd 1.66771

In [0]:
vals = np.zeros((len(df_limit_history),max_history))
logit_val = np.zeros((len(df_limit_history),max_history))
length = np.zeros((len(df_limit_history),max_history))
logit_length = np.zeros((len(df_limit_history),max_history))
for i in range(len(df_limit_history)):
  i_vec = np.zeros((num_nodes,1))
  for j in range( min(max_history, i)-1, 0, -1):
    vals[i,j] = np.linalg.norm(df_limit_history["history"][i][j])
    i_vec = i_vec + df_limit_history["history"][i][j]
    length[i,j] = np.linalg.norm(i_vec)
    logit_length[i,j] = np.linalg.norm(np.dot( np.reshape(i_vec, [1,-1]), out_pca) )
    logit_val[i,j] = np.linalg.norm( np.dot(np.reshape(df_limit_history["history"][i][j], [1,-1]), out_pca))


#plt.plot(length.mean(0))
plt.subplot(2,2,1)
plt.semilogy((vals.mean(0)))
plt.subplot(2,2,2)
plt.semilogy((length.mean(0)))

#plt.plot(length.mean(0))
plt.subplot(2,2,3)
plt.semilogy((logit_val.mean(0)))
plt.subplot(2,2,4)
plt.semilogy((logit_length.mean(0)))


In [0]:
vals = np.zeros((len(df_limit_history),max_history))
length = np.zeros((len(df_limit_history),max_history))
for i in range(len(df_limit_history)):
  i_vec = np.zeros((num_nodes,1))
  for j in range(max_history-1,0,-1):
    vals[i,j] = np.linalg.norm(df_limit_history["history"][i][j])
    i_vec = i_vec + df_limit_history["history"][i][j]
    length[i,j] = np.linalg.norm(i_vec)

len( df_limit_history["history"])
#plt.plot(length.mean(0))
plt.subplot(1,2,1)
plt.loglog((vals.mean(0)))
plt.subplot(1,2,2)
plt.semilogy((vals.mean(0)))


In [0]:
#Fig7
plt.subplot(1,2,1)
plt.semilogy((length.mean(0))[:-1])
plt.semilogy((vals.mean(0))[:-1])
plt.subplot(1,2,2)

plt.loglog((length.mean(0))[:-1])
plt.loglog((vals.mean(0))[:-1])

In [0]:
#q, r = np.linalg.qr(out_pca)
# h  = sess.run(source_rnn_cell_pca.zero_state(1, dtype=tf.float32))
# zero_state = h.dot(q)
# losses = []
# counter = 0
# temp = np.copy(b_h_pca)
# temp_offset = np.dot(b_h_pca,q)
# temp_matix = np.zeros((27,27,27))

# for i in range(27):
#   temp_matix[i] = np.dot(np.transpose(q), np.dot(w_h_pca_nxn[i],q))

test = np.zeros((num_nodes, num_nodes))
test[:,:27] = out_pca 

u,s,v = np.linalg.svd(test)
u[:,27].dot(out_pca)
basis = np.transpose(u)
basis_inv = np.linalg.inv(basis)

# out_pca = np.dot( (pca.components_), out_inn)
# init_pca = np.dot( (pca.components_), np.transpose(init))
# w_h_pca_nxn = np.zeros((27, pca_dim, pca_dim ))
# w_h_nxn =  np.zeros((27, num_nodes, num_nodes ))
# for i in range(27):
#   tmp = np.reshape(w_h_inn[i, :], [num_nodes, num_nodes] )
#   w_h_nxn[i] = tmp
#   tmp_2 = np.dot(pca.components_, np.dot(tmp, np.transpose(pca.components_)))
#   w_h_pca[i,:] = np.reshape( tmp_2, [-1]) 
#   w_h_pca_nxn[i] = tmp_2


#Barebones running of the RNN in reduced hidden state space of 27 x 27.
#h  = np.zeros((1, num_nodes))
h  = sess.run(source_rnn_cell_pca.zero_state(1, dtype=tf.float32))
losses = []
counter = 0
temp = np.copy(b_h_pca)
temp_offset = np.zeros((27, pca_dim)) 
temp_matix = np.zeros((27,pca_dim,pca_dim))
temp_out_pca = basis.dot(out_pca)
zero_state = basis.dot(np.transpose(h))

for i in range(27):
  temp_matix[i] = np.dot(basis, np.dot(w_h_pca_nxn[i],basis_inv))
  temp_offset[i] = np.dot(b_h_pca[i,:], np.transpose(basis)) #     np.transpose(np.dot(basis_inv, np.transpose(b_h_pca))) 
    

for i in range(10000,12000):
  if i % 50 == 0:
    h  = zero_state
  h_in = h
  input_val = np.argmax(x[i])
  h = np.dot( temp_matix[input_val], np.reshape( h, [-1,1] )) + np.reshape(temp_offset[input_val], [-1,1] ) 
  
  outpt_np = np.dot( np.reshape(h, [1,-1]),temp_out_pca) + out_original_bias
  loss = sess.run( [train_loss_np], feed_dict={output_placeholder: outpt_np, target_placeholder:y[i]})
  count += 1
  if i % 50 > 10:
    losses.append(loss)
print("produced {} random states".format(n_random))
print(np.mean(losses))
print len(df_states_pca)

#print(np.dot(np.transpose(q),q ))

#print(abs((out_inn - q.dot(r)))).mean()

In [0]:
#Barebones running of the RNN with all elements accessible in numpy.
#h  = np.zeros((1, num_nodes))
h  = sess.run(source_rnn_cell_pca.zero_state(1, dtype=tf.float32))
losses = []
counter = 0
temp = np.copy(b_h_pca)
if True:
  #temp[:,:] = 0
  offset_for_propagation =temp  
  offset_for_readout = b_h_pca 


for i in range(10000,12000):
  if i % 50 == 0:
    h  = sess.run(source_rnn_cell_pca.zero_state(1, dtype=tf.float32))
  h_in = h
  input_val = np.argmax(x[i])
  h = np.dot( w_h_pca_nxn[input_val], np.reshape( h, [-1,1] )) + np.reshape(offset_for_readout[input_val], [-1,1] ) 
  
  outpt_np = np.dot( np.reshape(h, [1,-1]), out_pca) + out_original_bias
  loss = sess.run( [train_loss_np], feed_dict={output_placeholder: outpt_np, target_placeholder:y[i]})
  h = h -  np.reshape(offset_for_readout[input_val], [-1,1] ) + np.reshape(offset_for_propagation[input_val], [-1,1] )
  count += 1
  if i % 50 > 10:
    losses.append(loss)
print("produced {} random states".format(n_random))
print(np.mean(losses))
print len(df_states_pca)

#can take out the offset for current step and get reduction to 2.0
#if taken out from the propagation, error goes up to 3.75

In [0]:
fig = plt.figure(figsize=(10,20))

w_out = np.transpose(out_pca)
w_inn = np.transpose(b_h_pca)
print w_out.shape
print w_inn.shape

u,s,v = np.linalg.svd(w_out)
temp = w_out.dot(np.transpose(v))
b_1_inv = np.transpose(v)
b_1 = np.linalg.inv(b_1_inv)
q_1 = w_out.dot(b_1_inv)[:27,:27]

block_1 = np.identity(num_nodes)
for i in range(27):
  q_1[i] = q_1[i] / np.linalg.norm(w_out[i])
block_1[:27,:27] = q_1



plt.subplot(6,1,1)
plt.imshow(w_out.dot(b_1_inv))

plt.subplot(6,1,2)
basis_1 = block_1.dot(b_1)
plt.imshow(w_out.dot(np.linalg.inv(basis_1)))

plt.subplot(6,1,3)
w_til_in = basis_1.dot(w_inn)
w_til_in_sub = w_til_in[27:,:]
u,s,v = np.linalg.svd(w_til_in_sub)

b_2 = np.transpose(u)
plt.imshow(np.transpose(b_2.dot(w_til_in_sub)))

block_b2 = np.identity(num_nodes)
block_b2[27:,27:]=b_2
q_2 = b_2.dot(w_til_in_sub)[:27,:27]

# for i in range(27):
#   q_2[i] = q_2[i] / (np.linalg.norm(w_inn[:,i])

block_2 = np.identity(num_nodes)
block_2[27:2*27,27:2*27] = q_2

plt.subplot(6,1,4)
plt.imshow(np.transpose(np.linalg.inv(block_2).dot(block_b2).dot(w_til_in)))

#basis_complete maps h to h_tilde, basis_complete.dot(h) = h_tilde
basis_complete = np.linalg.inv(block_2).dot(block_b2).dot(basis_1)

plt.subplot(6,1,5)
plt.imshow(np.transpose(basis_complete.dot(w_inn)),'bwr')


plt.subplot(6,1,6)
plt.imshow(w_out.dot(np.linalg.inv(basis_complete)),'bwr')

random_states_pca_transformed = np.zeros(random_states_pca.shape)
for i in range(len(random_states_pca)):
  random_states_pca_transformed[i] = (basis_complete).dot(np.transpose(random_states_pca[i]))

pca_2 = PCA(n_components=pca_dim - 54)
pca_2.fit(random_states_pca_transformed[:,54:])
#pca_states_transform = random_states_pca[:,54:]
#pca_states_transform.var(1).mean()
#pca_states_transformed = (basis_complete).dot(np.transpose(random_states_pca))
block_3 = np.identity(num_nodes)
block_3[54:,54:] = (pca_2.components_)
basis_complete = block_3.dot(np.linalg.inv(block_2)).dot(block_b2).dot(basis_1)


In [0]:
random_states_pca_transformed = np.zeros(random_states_pca.shape)
for i in range(len(random_states_pca)):
  random_states_pca_transformed[i] = (basis_complete).dot(np.transpose(random_states_pca[i]))

In [0]:
var = abs(random_states_pca_transformed).max(0)
basis_complete_2 = np.copy(basis_complete)

for i in range(pca_dim):
  basis_complete_2[i] = basis_complete[i] / var[i]

In [0]:
corr = np.corrcoef(np.transpose(basis_complete_2))
plt.imshow(corr, 'bwr')

In [0]:
print out_pca.shape

u,s,v = np.linalg.svd(out_pca)

outpt_np = np.dot( np.reshape(h, [1,-1]), out_pca)
u[:,27].dot(out_pca)
basis_2 = np.transpose(u)

print u.shape, s.shape, v.shape, out_pca.shape

In [0]:
temp.shape

In [0]:
temp[50:55, 0:5]

In [0]:
#tytyt2
# for i in range(29)"
fig = plt.figure(figsize=(30, 30))
for i in range(27):
  plt.subplot(14,2,i+1)
  plt.title(chars[i])
  

  #_ = plt.imshow(np.transpose( w_h_pca_nxn[i].dot(out_pca)), cmap ='bwr',vmin=-4,vmax=4)
  #_ = plt.imshow(np.transpose( w_h_nxn[i].dot(out_inn)), cmap ='bwr')#,vmin=-4,vmax=4)
  _ = plt.imshow(np.transpose( temp_matix[i].dot(temp_out_pca)), cmap ='bwr')#,vmin=-4,vmax=4)

In [0]:
all_state, _, next_val, input_val, suprise, input_indx, certainty = predict(" testing", 5, 12.0) #basis
print next_val

In [0]:
plt.subplot?


In [0]:

#Fig_COPY_1 / mast_COPY_plot
string = " approximation for this "
errors = np.zeros(50)

fig = plt.figure(figsize=(20, 8))

#h  = np.zeros((1, num_nodes))
h  = sess.run(source_rnn_cell_pca.zero_state(1, dtype=tf.float32))
history = []
history_cont = []
input_string = string
max_history = len(input_string)
out_history = np.ones((max_history, 27))

canvas = np.zeros((len(string)*27, 2*max_history + 4))
probabilities = np.zeros((len(string)*27,1))

# df_limit_history = pd.DataFrame()
#h  = sess.run(source_rnn_cell_pca.zero_state(1, dtype=tf.float32))
#history.append(h)
count = 0
for i in range(len(input_string)):
  h_in = h
  input_val = mapping[input_string[i]]
  sum_of_hm1_cont = np.zeros((num_nodes,1))
  for k in range(len(history)):
    new_h = np.dot( w_h_pca_nxn[input_val], np.reshape( history[k], [-1,1] ))
    history[k] = new_h
    sum_of_hm1_cont = sum_of_hm1_cont + new_h

  new_offset = np.reshape(b_h_pca[input_val], [-1,1])
  history.append(new_offset)
  h = sum_of_hm1_cont +  new_offset
  history_cont = []
  for k in range(len(history)):
    canvas[i*27 : (i+1) * 27,(k+1)*2:(k+2)*2] = np.reshape( np.dot( np.reshape(history[k], [1,-1]), out_pca), [-1,1])
    history_cont.append( np.dot( np.reshape(history[k], [1,-1]), out_pca) )
  outpt_np = np.dot( np.reshape(h, [1,-1]), out_pca) + out_original_bias
  probabilities[i*27 : (i+1) * 27] = np.reshape( soft_max(outpt_np) ,[-1,1])
  
  canvas[i*27 : (i+1) * 27,-2:]  =  np.reshape(outpt_np, [-1,1])
  canvas[i*27 : (i+1) * 27,-4:-2]  = np.reshape(out_original_bias,[-1,1])
  
#   d = {"index" : i, "state": h, "input":input_val, "state_in":h_in, "delta":h-h_in, "loss": loss, "history":history[:], "output": outpt_np, "props": soft_max(outpt_np), "history_cont": history_cont[:], "char":input_string[i] }
#   df_limit_history = df_limit_history.append(d, ignore_index=True)


fig = plt.figure(figsize=(20, 5))



plt.subplot(2,1,1)    
ax  = plt.imshow(np.transpose(canvas), 'jet', interpolation = 'nearest', vmin = -5, vmax = 10)
_=plt.xticks( np.arange(0,max_history*27,27) )


#fig.colorbar(ax)

plt.subplot(2,1,2)    
_ = plt.plot( probabilities) 
_ = plt.xticks( np.arange(0,max_history*27,27) )

for i in range(max_history):
  val =   np.argmax(probabilities[i*27 : (i+1) * 27])
  plt.text(i*27 + val, probabilities[i*27 + val] , chars[val])


In [0]:
hist = df_limit_history["history_cont"][3]

In [0]:
hist[0].shape, len(hist)

In [0]:
i = 5
print canvas.shape, len(string)
canvas[i*27:(i+1) * 27,:].shape

In [0]:
?np.random.uniform

In [0]:
index = 11
hist = df_limit_history["history_cont"][index]

hist_negative = [np.zeros(27) for i in xrange(len(hist))]
hist_positive = [np.zeros(27) for i in xrange(len(hist))]
for i in xrange(len(hist)):
  for j in xrange(len(hist[i][0])):
    if hist[i][0][j] < 0:
      hist_negative[i][j] = hist[i][0][j]     
    else:
      hist_positive[i][j] = hist[i][0][j]

ind = np.arange(27)    # the x locations for the groups
width = 0.3       # the width of the bars: can also be len(x) sequence

summed = np.zeros(27)
#positive_summed = np.zeros(27)

colors = [np.random.uniform(size =3) for i in xrange(len(hist))]

axes = []
for i in xrange(len(hist_negative)):
  p = plt.bar(ind, hist_negative[i], width,bottom = summed, color = colors[i])
  summed+=hist_negative[i]
  axes.append(p)

for i in xrange(len(hist_positive)):
  p2 = plt.bar(ind + width, hist_positive[i], width, bottom = summed, color = colors[i])
  summed+=hist_positive[i]

plt.ylabel('Scores')
plt.title('Scores by group and gender')
plt.xticks(ind + width/2.)
plt.yticks(np.arange(0, 1, 10))
plt.legend(axes, df_limit_history["char"].values)

plt.show()

In [0]:
q, r = np.linalg.qr(out_pca)

print(abs((out_pca - q.dot(r)))).mean()
#print(abs((np.identity - ))).mean()
temp_offset = np.dot(b_h_pca,q)
#print temp_offset.shape
#i = 0
#print np.dot(np.transpose(q), np.dot(w_h_pca_nxn[i],q)).shape

plt.plot(q.dot(np.transpose(q)).diagonal())

In [0]:
#this cell builds the bigram_freq and cond_probs dicts. 
#bigram_freq[(char1, char2)] contains the counts of the bigram char1,char2 (both integers in [0,26]
#cond_probs[(char1, char2)] contains prob(char2 | char1)


from collections import Counter
bigram_freq = Counter()

for i in xrange(len(x)-1):
  bigram_freq[(np.argmax(x[i]), np.argmax(x[i+1]))] +=1

cond_probs = Counter()

def sum_cond(char_dict, char):
  return np.sum([char_dict[(char, i)] for i in xrange(27)])

for char in xrange(27):
  total_count = sum_cond(bigram_freq, char)
  for char2 in xrange(27):
    cond_probs[(char, char2)] = float(bigram_freq[(char, char2)])/total_count
  print char, total_count, sum_cond(cond_probs, char)


In [0]:
outpt_np

In [0]:
w_out.shape, w_inn.shape, bias_output.shape

In [0]:
#plot the conditional probs for a given char
char = 0
bias_output = np.transpose(np.dot(w_out, w_inn)) + out_original_bias
plt.plot(soft_max(bias_output[char]), "r", label="soft_max(W_out * b_x)")
probs = [cond_probs[char, i] for i in xrange(27)]
plt.plot(probs, "g", label = "empirical")
plt.xlim(0, 26)
plt.xlabel("Output Character")
plt.ylabel("Conditional Probability")
plt.title("P(char | space)")
plt.legend(loc = 0)


In [0]:
#compare correlation with bigram probs vs input basis probs
fig = plt.figure(figsize=(10, 7))
corrs = []
corrs_uni = []
for char in xrange(27):
  probs_network = soft_max(bias_output[char])
  cond_p = [cond_probs[char,i] for i in xrange(27)]
  corr = np.corrcoef(probs_network, cond_p)[0][1]
  corr_unigram = np.corrcoef(cond_p, freq)[0][1]
  print corr, corr_unigram
  corrs.append(corr)
  corrs_uni.append(corr_unigram)
  
ax = plt.scatter(corrs_uni, corrs, c = range(27), cmap = "hsv",  s = 60)
plt.xlim(0, 1.0)
plt.ylim(0, 1.0)
plt.xlabel("unigram correlation")
plt.ylabel("offset correlation")
plt.title("offset vs unigram correlation")
x_pts = np.linspace(0, 1, 100)
plt.plot(x_pts, x_pts, "r")
cbar = plt.colorbar(ax)


In [0]:
print out_pca.shape, basis_5.shape

In [0]:
w_out_trans = w_out.dot(np.linalg.inv(basis_complete))
print w_out_trans.shape
w_in_trans = basis_complete.dot(w_inn)
print w_inn.shape

In [0]:


#Same plot
fig = plt.figure(figsize=(15, 7))

mapping = {}
for i in range(27):
  mapping[chars[i]] = i
  

freq = np.zeros(27)
for i in x:
  freq[np.argmax(i)] +=1
  
freq = freq / np.sum(freq)
                            

#outpt_np = np.dot( b_h_pca[:,:],out_pca) + out_original_bias

#temp = b_h_pca
#temp[:,30:] = 0
outpt_np = np.dot(b_[:27],temp_out_pca) + out_original_bias
#outpt_corrupt = np.dot(temp_offset[:,temp_out_pca) + out_original_bias


plt.subplot(1,2,1)
plt.title("Decoded prio vs actual")
val_x =soft_max(np.reshape(out_original_bias, [1,-1]))
print val_x.shape
print(np.sum(val_x, 1))
_ = plt.plot(val_x[0])

plt.plot(freq,'r')
plt.subplot(1,2,2)
plt.title("Decoded probabilities from offset")

for i in range(27):
  _ =plt.plot(soft_max(outpt_np[i]))
  print 'char ', chars[i], 'next ', chars[np.argmax(outpt_np[i])], 'corrupted ', chars[np.argmax(outpt_corrupt[i])]

plt.savefig("/tmp/prior_vs_actual.pdf")
 #Case study: "United States" vs "United Nations"

  

In [0]:
#compare correlation with bigram probs vs input basis probs
corrs = []
corrs_uni = []
for char in xrange(27):
  probs_network = soft_max(outpt_np[char])
  cond_p = [cond_probs[char,i] for i in xrange(27)]
  corr = np.corrcoef(probs_network, cond_p)[0][1]
  corr_unigram = np.corrcoef(cond_p, freq)[0][1]
  print corr, corr_unigram
  corrs.append(corr)
  corrs_uni.append(corr_unigram)
  
plt.scatter(corrs_uni, corrs, c = range(27), cmap = "hsv")
plt.xlim(0, 1.0)
plt.ylim(0, 1.0)
plt.xlabel("unigram correlation")
plt.ylabel("offset correlation")
plt.title("offset vs unigram correlation")
x_pts = np.linspace(0, 1, 100)
plt.plot(x_pts, x_pts, "r")

In [0]:
fig = plt.figure(figsize=(24, 10))

i = mapping['d']
plt.subplot(1,2,1)
plt.imshow(w_h_pca_nxn[i],'bwr')
plt.subplot(1,2,2)
w_h_trans = basis_complete_2.dot(w_h_pca_nxn[i]).dot(np.linalg.inv(basis_complete_2))
ax = plt.imshow(w_h_trans,'bwr', interpolation = 'nearest')
fig.colorbar(ax)
#transformed_imag=  

In [0]:
h_in = np.zeros((1, num_nodes))
all_state, _, next_val, input_val, suprise, input_indx, certainty = predict("g", 5, 10.0, h= h_in) #basis
print next_val

all_state, _, next_val, input_val, suprise, input_indx, certainty = predict("g", 5, 10.0) #basis
print next_val

In [0]:
#fi5
fig = plt.figure(figsize=(15, 10))

corrections = w_h_trans[0:27, 0:54]
ax = plt.imshow(corrections,'bwr', interpolation = 'none')
fig.colorbar(ax)
mapping['d']

#"x + E * x + e
for i in range(27):
  plt.text(27+(i-1.5)*1, 0, chars[i])
  plt.text(0, (i-1)*1.05, chars[i])

In [0]:
all_state, _, next_val, input_val, suprise, input_indx, certainty = predict("doctors wear scrubs and", 500, 1) #basis
print next_val

In [0]:
#Figure1
fig = plt.figure(figsize=(24, 8))



plt.subplot(3,2,1)
plt.title("Readout matrix in original")

plt.imshow(np.transpose(out_inn),interpolation = 'none',cmap ='bwr')

plt.subplot(3,2,2)
plt.title("Offset vectors in original")
plt.imshow((b_ixn), cmap = 'bwr', interpolation = 'none')


plt.subplot(3,2,3)
plt.title("Readout matrix in Pca")
plt.imshow(np.transpose(out_pca),interpolation = 'none',cmap ='bwr')

plt.subplot(3,2,4)
plt.title("Offset vectors in PCA")

plt.imshow((b_h_pca), cmap = 'bwr', interpolation = 'none')


plt.subplot(3,2,5)
plt.title("Readout in Readout basis")
plt.imshow(np.transpose(temp_out_pca), cmap = 'bwr', interpolation = 'none')



plt.subplot(3,2,6)
plt.title("Offset vectors in Readout basis")

#plt.imshow(w_h_pca_nxn[17][:,:], cmap = 'bwr', interpolation = 'none')
#plt.imshow(np.transpose(out_pca)>0,interpolation = 'none',cmap ='gray')
#plt.plot(np.sum(np.transpose(out_pca)**2,0))

plt.imshow((temp_offset), cmap = 'bwr', interpolation = 'none')

plt.savefig("/tmp/readout_and_offset_in_pca.pdf")

In [0]:
fig = plt.figure(figsize=(5, 5))
mat = np.zeros((10,10))
mat[:,1]=10
mat[:,2:3]=5
plt.imshow(mat, cmap = 'bwr', interpolation = 'none')



In [0]:
fig = plt.figure(figsize=(22, 16))

for i in range(27):
  plt.subplot(5,6, i + 1)
  plt.imshow(w_h_pca_nxn[i][20:180,20:180], cmap = 'bwr', interpolation = 'none')
  #plt.imshow(np.log(w_h_pca_nxn[i]**2), cmap = 'bwr', interpolation = None)

  #plt.imshow(w_h_nxn[i], cmap = 'bwr', interpolation = 'none')
  plt.title(chars[i])


In [0]:
#w_h_pca_nxn  --27 weights for transition
#out_pca  -- readout matrix 
#b_h_pca   --27 offset per character 
#out_original_bias --offset vector for readout. Explained: Predicts the average character probability 

#   w_h_nxn
#   b_ixn
#   out_inn
#   out_original_bias


  

plt.subplot(1,2,1)
lines = np.zeros((27,num_nodes))
for i in range(27):
  lines[i] =np.sum(w_h_pca_nxn[i]**2,0)
_ =plt.plot(np.mean((np.transpose(lines))**0.5,1))

plt.subplot(1,2,2)
lines = np.zeros((27,num_nodes))
for i in range(27):
  lines[i] =np.sum(w_h_pca_nxn[i]**2,1)
  
_ =plt.plot((((lines))**0.5))

In [0]:
test = np.zeros((num_nodes, num_nodes))
test[:,:27] = out_pca 

u,s,v = np.linalg.svd(test)

u[:,27].dot(out_pca)


In [0]:
corr = np.corrcoef(np.transpose(w_h_pca_nxn[5]), np.transpose(out_pca))
#corr = np.corrcoef((w_h_pca_nxn[0]), np.transpose(out_pca))
corr_2 = np.corrcoef(np.transpose(w_h_pca_nxn[26]), np.transpose(out_pca))

#tytyt

print out_pca.shape
w_h_pca_nxn[2].shape
plt.subplot(1,2,1)
plt.imshow(corr, cmap = 'gray')

plt.subplot(1,2,2)

_ = plt.hist((np.reshape(corr[num_nodes:, : num_nodes], [-1])), bins = 100)
_ = plt.hist(corr_2[num_nodes:, : num_nodes],  bins = 100, hold = True, alpha = 0.5)


In [0]:
fig = plt.figure(figsize=(16, 16))

# for i in range(27):
#   plt.subplot(27,1, i + 1)
#   plt.imshow(np.transpose(w_h_pca_nxn[i][:,-13:-1]), cmap = 'bwr', interpolation = 'none')
#   plt.title(chars[i])
  
  
_ = plt.hist(np.reshape(w_h_pca_nxn,[-1]),  bins = 200, hold = True, alpha = 0.5)
plt.yscale('log', nonposy='clip')

In [0]:
w_out.shape, b_ixn.shape, bias_output.shape

In [0]:
def entropy_probs(dist):
  return np.sum([-np.log2(p+1e-7)*p for p in dist])

In [0]:
entropy_probs([.125 for i in xrange(8)])

In [0]:
char = mapping['a']
cond_dist = [cond_probs[(char,i)] for i in xrange(27)]
plt.plot(cond_dist)
plt.xticks(range(27), chars)
print entropy_probs(cond_dist)
pass

def get_cond_dist(char, use_log = False):
  if use_log:
    return
  return [cond_probs[(char,i)] for i in xrange(27)]
def kl_prob(p, q):
  return np.sum([p[i] * np.log2(1e-6 + p[i]/(q[i] + 1e-6)) for i in xrange(len(p))])

In [0]:
seaborn.set_style("whitegrid")
#offset plot
fig = plt.figure(figsize=(20, 6))

norms = np.zeros(27)

for i in range(27):
  norms[i] =( np.linalg.norm(b_ixn[i]))
  #print chars[ i ] , np.linalg.norm(b_h_pca[i])

plt.subplot(1,3,1)
ax = plt.scatter(np.log(freq)/np.log(2),norms, c = np.arange(27), cmap = 'bwr', s = 80)
plt.title("Norm of offset vector vs entropy in the character")
plt.xlabel("bits of entropy")
plt.ylabel("norm of offset vector")

plt.subplot(1,3,2)
plt.title("readout of characters")

#next_probs = np.array([[cond_probs[(i,j)] for j in xrange(27)] for i in xrange(27)])
mat2 = np.array([[kl_prob(get_cond_dist(i), get_cond_dist(j)) for j in xrange(27)] for i in xrange(27)])
mat2 = 1-np.array([[np.corrcoef(get_cond_dist(i), get_cond_dist(j))[0][1] for j in xrange(27)] for i in xrange(27)])
ax = plt.imshow(mat2, cmap = 'gray', interpolation = 'nearest')
plt.xticks(range(27), chars)
plt.yticks(range(27), chars)
plt.grid(False)
plt.subplot(1,3,3)

cos_dist = np.array([[sp.spatial.distance.cosine(b_ixn[i], b_ixn[j]) for j in xrange(27)] for i in xrange(27)])
plt.imshow(-cos_dist, cmap = 'gray', interpolation = 'none')
plt.xticks(range(27), chars)
plt.yticks(range(27), chars)
plt.grid(False)



np.sum(mat2, axis = 0)
plt.savefig("/tmp/bits_of_entropy.pdf")

In [0]:
fig = plt.figure(figsize=(20, 7))
plt.subplot(2,1,1)
q, r = np.linalg.qr(out_inn)

#print(np.dot(np.transpose(q),q ))

#print(abs((out_inn - q.dot(r)))).mean()

plt.imshow(np.transpose(q),interpolation='none', cmap = 'bwr')
#plt.imshow((r),interpolation='none', cmap = 'bwr')
#plt.imshow((r),interpolation='none', cmap = 'bwr')
plt.subplot(2,1,2)

plt.plot(r.diagonal())

In [0]:

#What is the norm of the carried forward part

In [0]:
norms = ([np.linalg.norm(df_states_pca["state"][i]) for i in xrange(len(df_states_pca))])
p = plt.hist(norms, bins = 100)
print random_states_pca.shape
print np.mean(norms)


In [0]:
def scatter_plot(pd_input, color_map, do_pca = True):
  points = np.array([v[0] for v in pd_input.values])
  if do_pca:
    points = pca.transform(points)
  ax = plt.scatter(points[:,0], points[:,1], c = color_map, cmap = 'bwr' )
  return ax

In [0]:
df_limit_history["state"][0].shape

In [0]:
#PCA K-means plot using random states generated in the PCA dimension for the kmeans
k_means_dim = 1000
kmean = KMeans(k_means_dim, precompute_distances = False)
kmean.fit(random_states_pca)

max_history = 5

# #define the inputs
# index = range(n_random) 
# np.random.shuffle(index)
# inputs = np.zeros((27*num_each_char,27))
# for i in range(27*num_each_char):
#   inputs[i, i % 27] = 1

# This produces n_random hidden states by running the RNN in feed forward mode with the input. We could add a random sampling probability here.
df_states_kmeans = pd.DataFrame()
errors = np.zeros(50)
#h  = np.zeros((1, num_nodes))
h  = sess.run(source_rnn_cell_pca.zero_state(1, dtype=tf.float32))

losses = []

for i in range(30000,30000+5000):
  if i % 50 == 0:
    h  = sess.run(source_rnn_cell_pca.zero_state(1, dtype=tf.float32))
  h_in = h
  h = np.reshape(kmean.cluster_centers_[kmean.predict(h_in)[0]], [1,-1])
  h,loss = sess.run( [hidden_state_source_pca, train_loss_pca], feed_dict={input_placeholder:x[i], state_placeholder_pca:h, target_placeholder:y[i]})

  d = {"index" : i, "state": h, "input":np.argmax(x[i]), "state_in":h_in, "delta":h-h_in, "dist": i % 50, "loss": loss, "history":history[:] }
  df_states_kmeans = df_states_kmeans.append(d, ignore_index=True)
  if i % 50 > 13:
    losses.append(loss)
  errors
print("produced {} random states".format(n_random))
print(np.mean(losses))
print len(df_states)

In [0]:
#Transition matrix for all character input. Note that space bar almost resets. But not quite :)
fig = plt.figure(figsize=(12, 10))

inputs = np.zeros((27,27))
trans_x = []
trans_y = []
col = []
for i in range(27):
  inputs[i, i % 27] = 1
transition_matrix = np.zeros((27, k_means_dim,k_means_dim))
state_in = np.zeros((27, num_nodes))
for i in range(k_means_dim):
  for k in range(27):
    state_in[k] =  kmean.cluster_centers_[i]
  h_out = sess.run( [hidden_state_source_pca], feed_dict={input_placeholder:inputs, state_placeholder_pca:state_in})
  h_out_pred  = kmean.predict(h_out[0])
  for in_val in range(27):
    transition_matrix[k,i,h_out_pred[in_val]] = 1
    trans_x.append(h_out_pred[in_val])
    trans_y.append(i)
    col.append(in_val)
y_ax = plt.subplot(1,1,1)    
ax = plt.scatter(trans_x, trans_y, c = col, lw = 0,cmap = 'bwr', alpha = 0.5 ) 
cbar = fig.colorbar(ax)
y_ax.set_xlim(0, k_means_dim)
y_ax.set_ylim(0, k_means_dim)

plt.savefig("/tmp/finite_state_machine_transition_matrix.pdf")

In [0]:
#tytyt
ax = scatter_plot(df_states_pca["state"],df_states_pca["input"], False)
x_map = []
y_map = []
fix_vects = np.zeros((27, pca_dim))
for i in range(27):
  w_i = w_h_pca_nxn[i] 
  b_i = b_h_pca[i]
  _, vec = np.linalg.eig(w_i)
  fix = np.dot(np.linalg.inv(np.eye(pca_dim)-w_i),b_i)
  x_map.append(fix[0])
  y_map.append( fix[1])
  fix_vects[i] = fix
  
plt.scatter(x_map, y_map, c = np.arange(27),  cmap = 'bwr', s = 80)

inputs_t = np.zeros((1,27))
inputs_t[0,-1]=1
h = sess.run( [hidden_state_source_pca], feed_dict={input_placeholder:inputs_t, state_placeholder_pca:np.reshape(fix, [1,-1])})
print(np.sum(h-fix)**2)

plt.savefig("/tmp/all_fixed_points_in_pca.pdf")

In [0]:
#WARNING _ THIS IS PRETTY USELESS AS FAR AS I CAN TELL
#Free form sampling Starting at Fixed points
df_states_sampled = pd.DataFrame()
#h  = np.zeros((1, num_nodes))
h  = sess.run(source_rnn_cell.zero_state(1, dtype=tf.float32))
losses = []
input_t_saved = np.zeros((27, 27))

for i in range(27):
  input_t_saved[i, i] = 1
samples = []
for i in range(100):
  in_real = np.zeros((1,27))
  if i % 100 == 0:
    h  = fix_vects
    input_t = input_t_saved
#   if i % 100  < 50 : #50 == 0 or i < 50:
#     input_t = x[i]
#     in_real = x[i]
  out_eval, h, t = sess.run( [output_eval_pca, hidden_state_source_pca, output_source_pca], feed_dict={input_placeholder:input_t, state_placeholder_pca:h, target_placeholder:y[i], temperature: [2]})
  input_t = np.zeros((27, 27))
  for i in range(27):  
    choice = np.random.choice(len(out_eval[i,:]), p=out_eval[i])
    input_t[i, choice] = 1

  d = {"index" : i, "state": h, "input":np.argmax(x[i]), "output" : t, "input_vec":(in_real)}
  df_states_sampled = df_states_sampled.append(d, ignore_index=True)
  if i % 50 > 13:
    losses.append(loss)
print("produced {} random states".format(n_random))
print(np.mean(losses))

#train_task.print_text_input_output_targets(df_states_sampled["input_vec"].values, df_states_sampled["output"].values, df_states_sampled["output"].values)


for i in range(27):
  vals = []
  for k in  range(100):
    vals.append(df_states_sampled["output"][k][i:i+1])
  print chars[i], 'text', train_task.print_text_input_output_targets(df_states_sampled["input_vec"].values, df_states_sampled["input_vec"].values,vals)

    
      
      
  

In [0]:
#Fixed points for different charactes.

corr = np.corrcoef(fix_vects)
plt.subplot(1,4,1)
plt.title("fixed point corr")
corr = np.corrcoef(fix_vects)
plt.imshow(corr,cmap="gray",vmin=-1,vmax=1, interpolation='none' )
plt.subplot(1,4,2)
plt.title("off set Corre")
corr = np.corrcoef(b_h_pca)
plt.imshow(corr,cmap="gray",vmin=-1,vmax=1, interpolation='none' )
plt.subplot(1,4,3)
corr = np.corrcoef(np.transpose(b_h_pca))
x_ = plt.imshow(corr,cmap="gray",vmin=-1,vmax=1, interpolation='none' )
# summed = np.sum(corr**4,1)
# indx= np.argsort( summed)
# corr_2= np.corrcoef(np.transpose(b_h_pca[:,indx]))

plt.subplot(1,4,4)
#correlation of the read out matrix
#np.linalg.qr()

# all_mat = np.zeros((k_means_dim, k_means_dim))
# for i in range(27):
#   all_mat += transition_matrix[i]
# plt.imshow(all_mat,cmap="gray")

corr = np.corrcoef(np.transpose(out_inn))
_=plt.imshow(corr,cmap="gray",vmin=0,vmax=1, interpolation='none' )



In [0]:
fig = plt.figure(figsize=(10, 7))
plt.subplot(2,3,1)
_ =plt.plot((b_h_pca))


# w_h_pca = np.zeros((27, pca_dim**2))
# b_h_pca = np.zeros((27, pca_dim))
# out_pca = np.dot( (pca.components_), out_inn)
# init_pca = np.dot( (pca.components_), np.transpose(init))
plt.subplot(2,3,2)
plt.scatter(b_h_pca[:,3],b_h_pca[:,4], color= 'r')
plt.scatter(b_h_pca[:,5],b_h_pca[:,6],color = 'b')

plt.subplot(2,3,3)


plt.subplot(2,3,4)
_ =plt.plot(np.transpose(b_h_pca)[0:20,:])

plt.subplot(2,3,5)
_ =plt.plot(np.transpose(b_ixn))


plt.subplot(2,3,6)
_ =plt.plot(np.log(np.transpose(b_h_pca)[0:20,:]**2))

plt.savefig("/tmp/draft_plot_overview2.pdf")

In [0]:
plt.plot(gru308_ev, label = "gru-308")
plt.plot(gru145_ev, label = "gru-145")
plt.plot(rnn256_ev, label = "rnn-256")
plt.plot(pca.explained_variance_ratio_, label = "sls-216")
plt.ylim(1e-3, .2)
plt.xlim(0, 100)
plt.legend()

In [0]:
fig = plt.figure(figsize=(24, 7))

plt.subplot(1,3,1)
x_map = []
y_map = []
input_val = []
input_val_2 = []
all_ev  = []
all_all_ev = []
for k in range(27):
  jacob = w_h_pca_nxn[k]
  ev = np.linalg.eig(jacob)
  all_all_ev.append(ev[1])
  for j in range(num_nodes):
    x_map.append(ev[0][j].real)
    y_map.append(ev[0][j].imag)  
    input_val.append(j)
    input_val_2.append(k)
    if j < 10:
      all_ev.append(ev[1][:][j])



ax = plt.scatter(x_map, y_map, c = input_val,  cmap = 'bwr', s = 20,  lw = 0)
cbar = fig.colorbar(ax)

plt.subplot(1,3,2)

ax = plt.scatter(x_map, y_map, c = input_val_2,  cmap = 'bwr', s = 20,  lw = 0, alpha = 0.5)
cbar = fig.colorbar(ax)

plt.subplot(1,3,3)
corr = np.corrcoef(all_ev)
plt.imshow(np.abs(corr),cmap="gray",vmin=0,vmax=0.5)#, interpolation='none' )

plt.savefig("/tmp/spectrum_plot.pdf")

In [0]:
print pca.explained_variance_ratio_[:10]
print np.sum(pca.explained_variance_ratio_)

vals = np.zeros((len(df_limit_history),max_history))
for i in range(len(df_limit_history)):
  for j in range(max_history):
    vals[i,j] = np.linalg.norm(df_limit_history["history"][i][j])  

# x_val_decay = []
# y_val_decay = []
# vals_y = np.log(np.mean(vals,0))
# for i in range(len(vals_y)):
#   x_val_decay.append( np.log(i+1))
#   y_val_decay.append(vals_y[i])
#plt.subplot(1,2,1)
#plt.plot(x_val, y_val)
#plt.subplot(1,2,2)
#plt.plot(np.mean(vals,0),hold = True)
#plt.title("Norm of incoming state contribution after n steps")

np.sum(np.mean(vals,0))

fig = plt.figure(figsize=(22, 5))

x_val = []
y_val = []
col = []
count = 0
for k in range(27):
  for j in range(num_nodes):
    y_val.append( ((x_map[count]**2 + y_map[count]**2))**0.5)
    x_val.append(j)
    col.append(k)
    count = count + 1
plt.subplot(1,5,1)
plt.title("Norm of Eigenvalues")
ax = plt.scatter(x_val, y_val, c = col,  cmap = 'bwr', s = 10,  lw = 0)
cbar = fig.colorbar(ax)

norms = []
for k in range(27):
  norms.append(np.linalg.norm(w_h_pca_nxn[k]))
plt.subplot(1,5,2)
plt.title("Norm of Weightmatrices")

plt.plot(norms)

plt.subplot(1,5,3)
plt.title("Decay of norm over time")
#plt.plot(x_val_decay, y_val_decay)
plt.plot(np.mean(vals,0))
# dets  = []
# ranks = []
# for k in range(27):
#   dets.append(np.linalg.det(w_h_pca_nxn[k]))
#   ranks.append(np.linalg.matrix_rank(w_h_pca_nxn[k]))
# plt.plot(ranks)
# plt.plot(dets)


plt.subplot(1,5,4)
plt.title("First 20 dimensions of 5 EV")
dets  = []
for k in range(27):
  plt.plot(all_all_ev[k][:20,:5],hold = True)
  
plt.subplot(1,5,5)
plt.title("Correlation for offset vectors")

dets  = []
# for k in range(27):
#   plt.plot(np.transpose(all_all_ev[k][:10,:10]),hold = True)
corr = np.corrcoef((b_h_pca))
x_ = plt.imshow(corr,cmap="gray",vmin=-1,vmax=1, interpolation='none' )

plt.savefig("/tmp/eigen_val_and_decay.pdf")

In [0]:

fig = plt.figure(figsize=(22, 16))

for i in range(27):
  plt.subplot(5,6, i + 1)

  #corr = np.abs(np.corrcoef(np.transpose(b_h_pca)[0:20,:],(all_all_ev[k][:20,:20]), 0))
  #plt.imshow(corr,cmap="gray",vmin=-1,vmax=1, interpolation='none') #

  corr = np.abs(np.corrcoef(b_h_pca[:,:pca_dim],np.transpose(all_all_ev[i][:pca_dim,:30]), 1))
  plt.imshow(corr,cmap="gray",vmin=0,vmax=1, interpolation='none') #

plt.subplot(5,6, 2)

plt.title("correlation between offset vectors and 30 first eigenvectors")



In [0]:
#Below here is random stuff

In [0]:
k =10
matrx = w_h_pca_nxn[k]
evals, evects = np.linalg.eig(matrx)
ident = np.identity(num_nodes) 

eig_num = 30
#print np.dot(matrx - evals[eig_num]* ident, (evects[:,eig_num])) [0:3]

identity = np.identity(num_nodes)

ident = np.diag(evals)
  
print (np.dot((evects), np.dot(ident, np.linalg.inv(evects))) - matrx) [0:2,0:2]

In [0]:


fig = plt.figure(figsize=(8, 8))
ax = scatter_plot(df_states["state"], df_states["input"])
cbar = fig.colorbar(ax)

plt.savefig("/tmp/pca_plot_hidden_state_by_input.pdf")

In [0]:
kmean = KMeans(200)
kmean.fit(random_states[:8000,:])

In [0]:
kmean = KMeans(200)
kmean.fit(random_states[:8000,:])

max_history = 5

#define the inputs
index = range(n_random) 
np.random.shuffle(index)
inputs = np.zeros((27*num_each_char,27))
for i in range(27*num_each_char):
  inputs[i, i % 27] = 1

# This produces n_random hidden states by running the RNN in feed forward mode with the input. We could add a random sampling probability here.
df_states = pd.DataFrame()
errors = np.zeros(50)
#h  = np.zeros((1, num_nodes))
h  = sess.run(source_rnn_cell.zero_state(1, dtype=tf.float32))

losses = []

for i in range(10000,10000+5000):
  if i % 50 == 0:
    h  = sess.run(source_rnn_cell.zero_state(1, dtype=tf.float32))
  h_in = h
  h = np.reshape(kmean.cluster_centers_[kmean.predict(h_in)[0]], [1,-1])
  h,loss = sess.run( [hidden_state_source, train_loss], feed_dict={input_placeholder:x[i], state_placeholder:h, target_placeholder:y[i]})

  d = {"index" : i, "state": h, "input":np.argmax(x[i]), "state_in":h_in, "delta":h-h_in, "dist": i % 50, "loss": loss, "history":history[:] }
  df_states = df_states.append(d, ignore_index=True)
  if i % 50 > 13:
    losses.append(loss)
  errors
print("produced {} random states".format(n_random))
print(np.mean(losses))
print len(df_states)

In [0]:
#kmean.predict(random_states[:200,:])
data = pca.transform(kmean.cluster_centers_)
plt.scatter(data[:,0], data[:,1])



In [0]:


from sklearn.manifold import TSNE
def plot_tsne(data_points, labels):
  
  model = TSNE(n_components=2, random_state=0)
  np.set_printoptions(suppress=True)
  fit_tsne = model.fit_transform(data_points)

  def map_vals(vals):
    vals_set = list(set(vals))
    map_dict = {}
    for i, v in enumerate(vals_set):
      map_dict[v] = i
    return [map_dict[v] for v in vals]

  #print fit_tsne

  x = [h[0] for h in fit_tsne]
  y = [h[1] for h in fit_tsne]

  colors = map_vals(labels)
  print len(set(colors)) 

  p = plt.scatter(x + np.random.randn(len(x))*0.01,y,c = colors, cmap = "Set1")
  return x, y, colors
  
  
x_val, y_val, col = plot_tsne(random_states, df_states["input"])
  
  

In [0]:

#Free form sampling
df_states_sampled = pd.DataFrame()
#h  = np.zeros((1, num_nodes))
h  = sess.run(source_rnn_cell.zero_state(1, dtype=tf.float32))
losses = []

samples = []
for k in range(1000):
  i = (k // 100) * 4 + k
  in_real = np.zeros((1,27))
  if i % 100 == 0:
    h  = sess.run(source_rnn_cell.zero_state(1, dtype=tf.float32))
  if i % 100  < 50 : #50 == 0 or i < 50:
    input_t = x[i]
    in_real = x[i]
  out_eval, h,loss, t = sess.run( [output_eval, hidden_state_source, train_loss, output_source], feed_dict={input_placeholder:input_t, state_placeholder:h, target_placeholder:y[i], temperature: [10]})
  #out_eval[0][-1]= 1 -np.sum(out_eval[0][0:-1])
  #input_t = np.random.multinomial(1, out_eval, size=1)
  choice = np.random.choice(len(out_eval[0,:]), p=out_eval[0])
  input_t = np.zeros((1, 27))
  input_t[0, choice] = 1

  d = {"index" : i, "state": h, "input":np.argmax(x[i]), "output" : t, "input_vec":(in_real)}
  df_states_sampled = df_states_sampled.append(d, ignore_index=True)
  if i % 50 > 13:
    losses.append(loss)
print("produced {} random states".format(n_random))
print(np.mean(losses))

train_task.print_text_input_output_targets(df_states_sampled["input_vec"].values, df_states_sampled["output"].values, df_states_sampled["output"].values)


In [0]:
pca = PCA(n_components=2)
pca.fit(random_states)

stats ={}
stats['pca_explained_variance'] = pca.explained_variance_ratio_
print(stats)


#fixed_point_states = np.array([v for v in df_expansions["state"].values])
#approx_out_pca = pca.transform(fixed_point_states)

In [0]:
def get_energy(h_in, input_i, componentwise = False):
  inputs = np.zeros((1,27))
  h_in = np.reshape(h_in, [1, -1])
  inputs[0,input_i] = 1
  h = sess.run( hidden_state, feed_dict={input_placeholder:inputs, state_placeholder:h_in, expansion_point:h_in})
  coer = np.corrcoef(h, h_in)[1,0]
  if componentwise:
    return (h_in-h)**2, coer
  else:
    return (np.sum((h_in-h)**2, 1)), coer



In [0]:
pca = PCA(n_components=2)
pca.fit(random_states)
#plt.plot(random_states[0:100,:])

In [0]:
pca = PCA(n_components=2)
pca.fit(random_states)

fixed_point_states = np.array([v for v in df_expansions["state"].values])
approx_out_pca = pca.transform(fixed_point_states)


save_fig_path = "/tmp/"
fig = plt.figure(figsize=(22, 7))
bounds =range(len(df_expansions))
bounds = list(np.where(df_expansions["energy"][:] < 0.01))[0]
print len(bounds)
plots = []
plots.append(plt.subplot(2,3,1))
cax = plt.scatter(approx_out_pca[bounds,0],approx_out_pca[bounds,1], c = np.log(df_expansions["energy"][bounds])/np.log(10), cmap = "autumn" )
cbar = fig.colorbar(cax)

norm_states = np.zeros(len(random_states))
for i, state in enumerate( random_states):
  norm_states[i],_ = get_energy(np.reshape(state,[1,-1]), np.argmax(x[i]))
  
plots.append(plt.subplot(2,3,2))

all_states_pca = pca.transform(random_states)  
cax = plt.scatter(all_states_pca[:,0],all_states_pca[:,1], c = np.log(norm_states)/np.log(10), cmap = "autumn", hold = True )
cbar = fig.colorbar(cax) 

uniform_states =  np.zeros((len(random_states),num_nodes))
norm_states_uniform = np.zeros(len(random_states))
for i, state in enumerate( random_states):
  state_new = np.reshape( np.random.uniform(-1, 1, num_nodes), [1, -1])
  norm_states_uniform[i] = get_energy(state_new, np.argmax(x[i]))[0]
  uniform_states[i,:]=(state_new)[0]


plots.append(plt.subplot(2,3,3))
uniform_states_pca = pca.transform(uniform_states)  
cax = plt.scatter(uniform_states_pca[:,0],uniform_states_pca[:,1], c = np.log(norm_states_uniform)/np.log(10), cmap = "autumn", hold = True )
cbar = fig.colorbar(cax) 

# norm = np.zeros(len(df_new["state"]))
# for i, state in enumerate( df_new["state"]):
#   norm[i] =sum(state**2)**0.5

# cax = plt.scatter(approx_out_pca[bounds,0],approx_out_pca[bounds,1], c = norm[bounds], cmap = "autumn" )
# cbar = fig.colorbar(cax)

# plt.subplot(2,2,1)


# # all_states_pca = pca.transform(random_states)  
# # cax = plt.scatter(all_states_pca[:,0],all_states_pca[:,1], c = norm_states, cmap = "autumn", hold = True )
# # cbar = fig.colorbar(cax) 
# for i, plot in enumerate(plots):
#   plot.set_xlim([-8,6])
#   plot.set_ylim([-8,6])
  
save_fig_path + "overview_plt"

plt.savefig(save_fig_path + "overview_plt" + FLAGS.source_rnn + "_num_" + str(num) + "_reg_" + reg + snapshop +  ".pdf")

In [0]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

#This might be a good plot - u and r by energy for fixed point and samples points
# h_t_bxn = u_bxn * h_tm1_bxn  + (1.0-u_bxn) * c_t_bxn
pre_activations = source_rnn_cell._pre_activations
mean_act = []
energy  = []
mean_r = []
bounds = list(np.where(df_expansions["energy"][:] < 0.001))[0]
number_to_plot = 10000
for k in range(number_to_plot):
  i = bounds[index[k] % len(bounds)]
  state  = fixed_point_states[i:i+1,:]
  input_val = df_expansions["input"][i]
  inputs = np.zeros((1,27))
  inputs[0,input_val]=1
  tensors = [pre_activations["fg_bias"], pre_activations["xh_t_bxipn"],pre_activations["r_act_bxn"],pre_activations["u_act_bxn"],pre_activations["xrh_t_bxipn"], pre_activations["c_act_bxn"]]
  tensors_eval = sess.run(tensors,  {input_placeholder:inputs, state_placeholder: state})
  mean_act.append(np.mean(sigmoid(tensors_eval[3][0]+forget_gate_hack)))
  mean_r.append(np.mean(sigmoid(tensors_eval[2][0])))  
  energy.append(df_expansions["energy"][i])

fig = plt.figure(figsize=(22, 7))

plots = []
plots.append(plt.subplot(1,3,1))
plt.title("Fixed points")
plt.xlabel("mean act u")
plt.ylabel("mean act r")

cax = plt.scatter(mean_act,mean_r, c = np.log(energy)/np.log(10),alpha = 0.2, cmap = "autumn",lw = 0  )
cbar = fig.colorbar(cax) 


mean_act_states = []
mean_energy_states  = []
mean_r_states = []

for i, state in enumerate(random_states):
  inputs = np.zeros((1,27))
  input_val = np.argmax(x[i])
  inputs[0,input_val]=1
  state  = random_states[i:i+1,:]
  tensors = [pre_activations["fg_bias"], pre_activations["xh_t_bxipn"],pre_activations["r_act_bxn"],pre_activations["u_act_bxn"],pre_activations["xrh_t_bxipn"], pre_activations["c_act_bxn"]]
  tensors_eval = sess.run(tensors,  {input_placeholder:inputs, state_placeholder: state})
  mean_act_states.append(np.mean(sigmoid(tensors_eval[3][0]+forget_gate_hack)))
  mean_r_states.append(np.mean(sigmoid(tensors_eval[2][0])))
  energy_new, _ = get_energy( state, input_val)
  mean_energy_states.append(energy_new)
  
plots.append(plt.subplot(1,3,2))
cax = plt.scatter(mean_act_states,mean_r_states, c = np.log(mean_energy_states)/np.log(10),alpha = 0.2, cmap = "autumn" ,lw = 0 )
cbar = fig.colorbar(cax) 

plt.title("Random States from trajectory")
plt.xlabel("mean act u")
plt.ylabel("mean act r")

mean_act_states = []
mean_energy_states  = []
mean_r_states = []

for i, state in enumerate(random_states):
  inputs = np.zeros((1,27))
  input_val = np.argmax(x[i])
  inputs[0,input_val]=1
  state  = np.reshape( np.random.uniform(-1, 1, num_nodes), [1, -1])
  tensors = [pre_activations["fg_bias"], pre_activations["xh_t_bxipn"],pre_activations["r_act_bxn"],pre_activations["u_act_bxn"],pre_activations["xrh_t_bxipn"], pre_activations["c_act_bxn"]]
  tensors_eval = sess.run(tensors,  {input_placeholder:inputs, state_placeholder: state})
  mean_act_states.append(np.mean(sigmoid(tensors_eval[3][0]+forget_gate_hack)))
  mean_r_states.append(np.mean(sigmoid(tensors_eval[2][0])))
  energy_new, _ = get_energy( state, input_val)
  mean_energy_states.append(energy_new)
  
plots.append(plt.subplot(1,3,3))
cax = plt.scatter(mean_act_states,mean_r_states, c = np.log(mean_energy_states)/np.log(10),alpha = 0.2, cmap = "autumn" ,lw = 0 )
cbar = fig.colorbar(cax) 

plt.title("Random States from all space")
plt.xlabel("mean act u")
plt.ylabel("mean act r")
for i, plot in enumerate(plots):
  plot.set_xlim([0,1])
  plot.set_ylim([0,1])

  
  
plt.savefig(save_fig_path + "overview_plt" + FLAGS.source_rnn + "_num_" + str(num) + "_reg_" + reg + snapshop +  ".pdf")

In [0]:
forget_gate_hack

In [0]:
mean_act_states = []
mean_energy_states  = []
mean_r_states = []
number_to_plot = 1000
plots = []
for k in range(number_to_plot):
  i = index[k]%len(df_expansions["input"])
  inputs = np.zeros((1,27))
  input_val = df_expansions["input"][i]
  inputs[0,input_val]=1
  state  = fixed_point_states[i:i+1,:]

  tensors = [pre_activations["fg_bias"], pre_activations["xh_t_bxipn"],pre_activations["r_act_bxn"],pre_activations["u_act_bxn"],pre_activations["xrh_t_bxipn"], pre_activations["c_act_bxn"]]
  tensors_eval = sess.run(tensors,  {input_placeholder:inputs, state_placeholder: state})
  energy_new, _ = get_energy( state, input_val, componentwise = componentwise)
  for k in range(num_nodes):
    mean_energy_states.append(energy_new[0][k % len(energy_new[0])])
    mean_act_states.append((tensors_eval[3][0]+6)[k])
    mean_r_states.append(k)  
plots.append(plt.subplot(1,3,1) )   
cax = plt.scatter(mean_r_states, mean_act_states, c = np.log(mean_energy_states)/np.log(10),alpha =0.1, cmap = "autumn",lw = 0)  
cbar = fig.colorbar(cax) 

mean_act_states = []
mean_energy_states  = []
mean_r_states = []
number_to_plot = 400
for k in range(number_to_plot):
  i = index[k]
  inputs = np.zeros((1,27))
  input_val = np.argmax(x[i])
  inputs[0,input_val]=1
  state  = np.reshape( np.random.uniform(-1, 1, num_nodes), [1, -1])
  state  = random_states[i:i+1,:]

  tensors = [pre_activations["fg_bias"], pre_activations["xh_t_bxipn"],pre_activations["r_act_bxn"],pre_activations["u_act_bxn"],pre_activations["xrh_t_bxipn"], pre_activations["c_act_bxn"]]
  tensors_eval = sess.run(tensors,  {input_placeholder:inputs, state_placeholder: state})
  energy_new, _ = get_energy( state, input_val, componentwise = componentwise)
  for k in range(num_nodes):
    mean_energy_states.append(energy_new[0][k % len(energy_new[0])])
    mean_act_states.append((tensors_eval[3][0]+6)[k])
    mean_r_states.append(k)  
plots.append(plt.subplot(1,3,2))    
cax = plt.scatter(mean_r_states, mean_act_states, c = np.log(mean_energy_states)/np.log(10),alpha =0.1, cmap = "autumn",lw = 0)  
cbar = fig.colorbar(cax) 
saved_u = mean_act_states

mean_act_states = []
mean_energy_states  = []
mean_r_states = []
number_to_plot = 400
for k in range(number_to_plot):
  i = index[k]
  inputs = np.zeros((1,27))
  input_val = np.argmax(x[i])
  inputs[0,input_val]=1
  state  = np.reshape( np.random.uniform(-1, 1, num_nodes), [1, -1])

  tensors = [pre_activations["fg_bias"], pre_activations["xh_t_bxipn"],pre_activations["r_act_bxn"],pre_activations["u_act_bxn"],pre_activations["xrh_t_bxipn"], pre_activations["c_act_bxn"]]
  tensors_eval = sess.run(tensors,  {input_placeholder:inputs, state_placeholder: state})
  energy_new, _ = get_energy( state, input_val, componentwise = componentwise)
  for k in range(num_nodes):
    mean_energy_states.append(energy_new[0][k % len(energy_new[0])])
    mean_act_states.append((tensors_eval[3][0]+6)[k])
    mean_r_states.append(k)  
plots.append(plt.subplot(1,3,3))    
cax = plt.scatter(mean_r_states, mean_act_states, c = np.log(mean_energy_states)/np.log(10),alpha =0.1, cmap = "autumn",lw = 0)  
cbar = fig.colorbar(cax) 

#for i, plot in enumerate(plots):
  #plot.set_xlim([0,num_nodes])
  #plot.set_ylim([-20,20])


In [0]:
bins_energy = np.arange(-80,45,0.1)
p2 = plt.hist(mean_act_states, bins=bins_energy, alpha = 1) #includes everything. It's the incoming activation to the U gate. Random suff, -1 to 1 in each dimension
p2 = plt.hist(saved_u, bins=bins_energy, alpha = 1, color = 'g') #includes everything. It's the incoming activation to the U gate. Random suff, -1 to 1 in each dimension


In [0]:
char = 1
constant_input = np.zeros((batch_size_fp * 27,27))
constant_input[:,char] = 1
#expansion_point_vals = np.reshape(sess.run(fixed_point_rnn_cell.expansion_points),[ 270, -1])
for i in range(10):
  expansion_state_val = np.reshape(state_input,[1,-1])  # expansion_point_vals[i:i+1,:]
  state = starting_states[i:i+1,:]
  approx_out, true_out = sess.run([output, hidden_state_true], feed_dict={input_placeholder:constant_input[i:i+1,:], state_placeholder:state, expansion_point: expansion_state_val  })


  pca = PCA(n_components=2)
  pca.fit(random_states)
  approx_out_pca = pca.transform(approx_out[0])
  true_out_pca = pca.transform(true_out)

  expansion_pca = pca.transform(expansion_state_val)
  state_pca = pca.transform(state)

  plt.scatter(approx_out_pca[:,0],approx_out_pca[:,1]) 

  plt.scatter(state_pca[:,0],state_pca[:,1], c = 'g', hold = True)
  plt.scatter(true_out_pca[:,0],true_out_pca[:,1], c = 'r', hold = True) 
  plt.scatter(expansion_pca[:,0], expansion_pca[:,1], c = 'y', hold = True, s = 80)
  plt.plot([expansion_pca[:,0],state_pca[:,0]], [expansion_pca[:,1],state_pca[:,1]], c = 'y', hold = True)
  
  plt.plot([state_pca[:,0],true_out_pca[:,0]],[state_pca[:,1],true_out_pca[:,1]], c = 'r', hold = True) 
  plt.plot([state_pca[:,0],approx_out_pca[:,0]],[state_pca[:,1],approx_out_pca[:,1]], c = 'b', hold = True, alpha = 0.5) 
  
  
  approx_out_pca = pca.transform(approx_out[1])
  plt.scatter(approx_out_pca[:,0],approx_out_pca[:,1],c ='k',s = 160) 




In [0]:
char = 1 
print df_states[df_states["input"]==1]["state"][0:1]
         
print(df_expansions["energy"].min())

best = df_expansions[df_expansions["input"] == 1]["energy"].argmin()
tmp = df_expansions[df_expansions["input"] == 1]

print tmp["energy"][best]
state_input = np.reshape(tmp["state"][best], [1,-1])
print get_energy(state_input, 1)
print best

In [0]:
df_expansions = pd.read_pickle("/tmp/results_csv_huge_fp_new" + FLAGS.source_rnn + ".pkl")
print len(df_expansions)


In [0]:
dist = []
for k in range(5):
  np.random.shuffle(index)
  for m in range(4):
    i = index[k]
    state = random_states[i:i+1,:] 
    expansion_state_val = state + np.random.randn(1,num_nodes)*0.4
    dist.append( np.sum((expansion_state_val-state)**2))
    approx_out, true_out = sess.run([output, hidden_state_true], feed_dict={input_placeholder:x[i], state_placeholder:state, expansion_point: expansion_state_val  })


    pca = PCA(n_components=2)
    pca.fit(random_states)
    approx_out_pca = pca.transform(approx_out[0])
    true_out_pca = pca.transform(true_out)

    expansion_pca = pca.transform(expansion_state_val)
    state_pca = pca.transform(state)

    plt.scatter(approx_out_pca[:,0],approx_out_pca[:,1], hold = True) 

    plt.scatter(state_pca[:,0],state_pca[:,1], c = 'g', hold = True)
    plt.scatter(true_out_pca[:,0],true_out_pca[:,1], c = 'r', hold = True) 
    plt.scatter(expansion_pca[:,0], expansion_pca[:,1], c = 'y', hold = True, s = 50)
    plt.plot([expansion_pca[:,0],state_pca[:,0]], [expansion_pca[:,1],state_pca[:,1]], c = 'y', hold = True)

    plt.plot([state_pca[:,0],true_out_pca[:,0]],[state_pca[:,1],true_out_pca[:,1]], c = 'r', hold = True) 
    plt.plot([state_pca[:,0],approx_out_pca[:,0]],[state_pca[:,1],approx_out_pca[:,1]], c = 'b', hold = True, alpha = 0.5) 


print(np.mean(dist))


In [0]:
df_distance_plot = pd.DataFrame()
distances = np.arange(0,50,2)
for d in distances:
  np.random.shuffle(index)
  for k in range(50):
    i = index[k]
    state = random_states[i:i+1,:] 
    expansion_state_val = state + np.random.randn(1,num_nodes)*((d)**0.5) /( num_nodes )**0.5
    distance = np.sum((expansion_state_val-state)**2)
    approx_out, true_out = sess.run([output, hidden_state_true], feed_dict={input_placeholder:x[i], state_placeholder:state, expansion_point: expansion_state_val })
    cos_err = sp.spatial.distance.cosine(approx_out[0], true_out)
    err = np.sum((true_out-approx_out[0])**2)
    energy_expansion = np.sum((expansion_state_val-approx_out[1])**2)
    energy_state = np.sum((state-true_out)**2)

    d_entry = {"cos_err" : cos_err, "err": err, "input":np.argmax(x[i]), "energy_expansion":energy_expansion, "distance":distance, "energy_state":energy_state}
    df_distance_plot = df_distance_plot.append(d_entry, ignore_index=True)
    

In [0]:
plt.subplot(1,2,1)
plt.scatter(df_distance_plot['distance'], df_distance_plot['cos_err'],c =  np.log(df_distance_plot['energy_expansion']), cmap = 'autumn')

plt.subplot(1,2,2)
plt.scatter(df_distance_plot['distance'], df_distance_plot['energy_expansion'],c =  np.log(df_distance_plot['err']), cmap = 'autumn')

In [0]:
# 
# 

In [0]:
#componentwise plot of above - 1 point per dimension
def add_np_arrays(x,y):
  x = np.concatenate([x,y],1)

pre_activations = source_rnn_cell._pre_activations
mean_act = np.array([])
energy  = np.array([])
mean_r =  np.array([])
inputs = np.zeros((1,27))
inputs[0,0]=1
for i, state in enumerate(res_states):
  state  = res_states[i:i+1,:]
  tensors = [pre_activations["fg_bias"], pre_activations["xh_t_bxipn"],pre_activations["r_act_bxn"],pre_activations["u_act_bxn"],pre_activations["xrh_t_bxipn"], pre_activations["c_act_bxn"]]
  tensors_eval = sess.run(tensors,  {input_placeholder:inputs, state_placeholder: state})
  add_np_arrays(mean_act, sigmoid(tensors_eval[3][0]))
  add_np_arrays(mean_r,sigmoid(tensors_eval[2][0])) 
  add_np_arrays(energy,(get_energy(state,10, componentwise = True))

fig = plt.figure(figsize=(18, 5))

plots = []
plots.append(plt.subplot(1,3,1))
plt.title("Fixed points")
plt.xlabel("mean act u")
plt.ylabel("mean act r")

cax = plt.scatter(mean_act,mean_r, c = np.log(energy)/np.log(10),alpha = 0.2, cmap = "autumn"  )
cbar = fig.colorbar(cax) 


mean_act_states = []
mean_energy_states  = []
mean_r_states = []

for i, state in enumerate(random_states):
  inputs = np.zeros((1,27))
  input_val = np.argmax(x[i])
  inputs[0,input_val]=1
  state  = random_states[i:i+1,:]
  tensors = [pre_activations["fg_bias"], pre_activations["xh_t_bxipn"],pre_activations["r_act_bxn"],pre_activations["u_act_bxn"],pre_activations["xrh_t_bxipn"], pre_activations["c_act_bxn"]]
  tensors_eval = sess.run(tensors,  {input_placeholder:inputs, state_placeholder: state})
  mean_act_states.append((sigmoid(tensors_eval[3][0])))
  mean_r_states.append((sigmoid(tensors_eval[2][0])))
  energy_new, _ = get_energy( state, input_val, componentwise = True)
  mean_energy_states.append(energy_new)
  
plots.append(plt.subplot(1,3,2))
cax = plt.scatter(mean_act_states,mean_r_states, c = np.log(mean_energy_states)/np.log(10),alpha = 0.2, cmap = "autumn"  )
cbar = fig.colorbar(cax) 

plt.title("Random States from trajectory")
plt.xlabel("mean act u")
plt.ylabel("mean act r")

mean_act_states = []
mean_energy_states  = []
mean_r_states = []

for i, state in enumerate(random_states):
  inputs = np.zeros((1,27))
  input_val = np.argmax(x[i])
  inputs[0,input_val]=1
  state  = np.reshape( np.random.uniform(-1, 1, num_nodes), [1, -1])
  tensors = [pre_activations["fg_bias"], pre_activations["xh_t_bxipn"],pre_activations["r_act_bxn"],pre_activations["u_act_bxn"],pre_activations["xrh_t_bxipn"], pre_activations["c_act_bxn"]]
  tensors_eval = sess.run(tensors,  {input_placeholder:inputs, state_placeholder: state})
  mean_act_states.append((sigmoid(tensors_eval[3][0])))
  mean_r_states.append((sigmoid(tensors_eval[2][0])))
  energy_new, _ = get_energy( state, input_val, componentwise = True)
  mean_energy_states.append(energy_new)
  
plots.append(plt.subplot(1,3,3))
cax = plt.scatter(mean_act_states,mean_r_states, c = np.log(mean_energy_states)/np.log(10),alpha = 0.2, cmap = "autumn"  )
cbar = fig.colorbar(cax) 

plt.title("Random States from all space")
plt.xlabel("mean act u")
plt.ylabel("mean act r")
for i, plot in enumerate(plots):
  plot.set_xlim([0,1])
  plot.set_ylim([0,1])

  
  
#Should add one more plot: Random staets from space
#How does speed increase as you start sampling from the inference path? 
#How does does speed relate to x-entropy
#How does speed increase as a function of radius away from the inference path and from random point? 
#How does Taylor expansion error increase as function of radius away from inference path and from random point?
#For given radius, what's the correlation between taylor expansion error and speed of the random point
#How does speed developt with randomly shuffled dimensions along the inference path
#What is the volume of the manifold that the inference path lies on? 

In [0]:
# This measure speed of random samples in the space of all points.
energies_random = []
coer_random = []
for i in range(10000):
  state_in = np.reshape( np.random.uniform(-1, 1, num_nodes), [1, -1])
  input_val = np.argmax(x[index[i]])
  energy, coer = get_energy(state_in, input_val)
  energies_random.append(energy)
  coer_random.append(coer)
  if i % 2000 == 0:
    print( i )
    
energies_sampled = []
coer_sampled = []
#h  = np.zeros((1, num_nodes))
for i in range(10000):
  state_in = random_states[index[i]:index[i]+1,:]
  input_val = np.argmax(x[index[i]])
  energy, coer = get_energy(state_in, input_val)
  energies_sampled.append(energy) 
  coer_sampled.append(coer)
  if i % 2000 == 0:
    print( i )
  
axs = []
bins_energy =get_bins([np.log(energies_random)/np.log(10), np.log(energies_sampled)/np.log(10)],100)
axs.append( plt.subplot(1,2,1))
axs.append( plt.subplot(1,2,2))
p1 = axs[0].hist(np.log(energies_random) / np.log(10), bins=bins_energy)
p2 = axs[0].hist(np.log(energies_sampled) / np.log(10), bins=bins_energy)

bins_coer =get_bins([coer_random, coer_sampled],100)


In [0]:
fig = plt.figure(figsize=(16, 7))

axs = []

val_random = np.log(energies_random)/np.log(10)
val_sampled = np.log(energies_sampled)/np.log(10)

# val_random = energies_random
# val_sampled = energies_sampled

bins_energy =get_bins([val_random, val_sampled],100)
axs.append( plt.subplot(1,2,1))
axs.append( plt.subplot(1,2,2))
p1 = axs[0].hist(val_random, bins=bins_energy)
p2 = axs[0].hist(val_sampled, bins=bins_energy,alpha = 0.7, color ='g')

bins_coer =get_bins([coer_random, coer_sampled],100)

p1 = axs[1].hist(coer_random, bins=bins_coer)
p2 = axs[1].hist(coer_sampled, bins=bins_coer,alpha = 0.7, color ='g' )

In [0]:
# This measure speed of random samples in the space of all points.
#energies_random = []
#coer_random = []
for i in range(1000000):
  state_in = np.reshape( np.random.uniform(-1, 1, num_nodes), [1, -1])
  input_val = np.argmax(x[index[i%10000]])
  energy, coer = get_energy(state_in, input_val)
  energies_random.append(energy)
  coer_random.append(coer)
  if i % 50000 == 0:
    print( i )


In [0]:
def get_bins(arrays, n):
  curr_min = np.min(arrays[0])
  curr_max = np.max(arrays[0])
  for a in arrays:
    curr_min = np.min([np.min(a), curr_min])
    curr_max = np.max([np.max(a), curr_max])
  return np.arange(curr_min, curr_max, (curr_max - curr_min) / n)
                   
    

In [0]:
plt.scatter(energies_random, coer_random, alpha = 0.4)
plt.scatter(energies_sampled, coer_sampled, c = 'r', hold = True, alpha = 0.4)

In [0]:
#rest of notebook is random stuff

In [0]:
#tytyt
#error with proximity penalty
sess.run(approx_error_weight.assign([0]))
sess.run(fp_penalty_weight.assign([1]))
sess.run(proximity_penalty_weight.assign([0.001]))

batch_size_fp = 10 
#start the distance by energy plot
df_expansions_prox = pd.DataFrame()
#num_trials = 1
num_trials = 1 
for k in range(num_trials):
  np.random.shuffle(index)

  starting_states = random_states[index[:batch_size_fp*27],:] + np.random.randn(batch_size_fp*27,num_nodes)*0.00001
  print starting_states.shape
  _ = sess.run( fixed_point_rnn_cell.expansion_points.assign( np.reshape(starting_states, [batch_size_fp, 27, -1])))

  constant_input = np.zeros((batch_size_fp * 27,27))
  constant_input[:,char] = 1
  bounds = np.ones(batch_size_fp * 27)
  #First use gradients
  old_loss = 1000
  for i in range(1000):
    _, loss_val, error_in_dim, expansion_points_eval  = sess.run( [optimizer_2, loss_2, outputs_t, fixed_point_rnn_cell.expansion_points] , feed_dict={input_placeholder:constant_input, state_placeholder:starting_states })
    expansion_points_eval = np.reshape(expansion_points_eval, [batch_size_fp * 27,-1])
    error_in_dim = np.sum(error_in_dim**2,1)
    if i % 100 == 0:
      print i
      for j in range(len( error_in_dim)): 
        energy = error_in_dim[j]
        dist = np.sqrt(np.sum((starting_states[j,:] - expansion_points_eval[j,:])**2))
        d = {"input" : char, "state": expansion_points_eval[j], "energy":energy, "dist":dist, "step":i}
        df_expansions_prox = df_expansions_prox.append(d, ignore_index=True)
    if i % 1000 == 0:
      print i,loss_val, np.mean(error_in_dim), np.mean(df_expansions_prox["dist"])
  #df_expansions.to_pickle("/tmp/results_csv_huge_fp_new.pkl") 
  print "len(df_expansions_prox): {}".format(len(df_expansions_prox))

In [0]:
filtered= df_expansions_prox["step"]>0
plt.plot(df_expansions_prox[filtered]["dist"].values, df_expansions_prox[filtered]["energy"].values,'r.')


In [0]:
constant_input = np.zeros((batch_size_fp * 27,27))
for i in range(270):
  constant_input[:,i%27] = 1
df_expansions = pd.DataFrame()
num_trials = 10000
for k in range(num_trials):
  np.random.shuffle(index)

  starting_states = random_states[index[:batch_size_fp*27],:] + np.random.randn(batch_size_fp*27,num_nodes)*0.1*(k/num_trials)
  print starting_states.shape
  _ = sess.run( fixed_point_rnn_cell.expansion_points.assign( np.reshape(starting_states, [batch_size_fp, 27, -1])))

  bounds = np.ones(batch_size_fp * 27)
  #First use gradients
  old_loss = 1000
  for i in range(20000):
    _, loss_val, error_in_dim, expansion_points_eval  = sess.run( [graph_dict['optimizer'],graph_dict['loss'], outputs_t, fixed_point_rnn_cell.expansion_points] , feed_dict={input_placeholder:constant_input})
    expansion_points_eval = np.reshape(expansion_points_eval, [batch_size_fp * 27,-1])
    error_in_dim = np.sum(error_in_dim**2,1)
    if i % 10 == 0:
      for j, val in enumerate( error_in_dim): 
        if val < bounds[ j ]:
          bounds[ j ] = bounds[ j ] /10
          dist = np.sqrt(np.sum((expansion_points_eval[j] - starting_states[j])**2))
          d = {"input" : j%27, "state": expansion_points_eval[j], "energy":val, "fake" : 0, "starting_state":starting_states[j], "step":i, "dist":dist}
          df_expansions = df_expansions.append(d, ignore_index=True)
      if np.abs(old_loss - loss_val) < .00001: #Note, I lowered this value
        print "converged to loss {}".format(loss_val)
        print d
        break
      old_loss = loss_val
    if i % 1000 == 0:
      print i,loss_val, np.mean(error_in_dim)
  #df_expansions.to_pickle("/tmp/results_csv_huge_fp_new" + FLAGS.source_rnn + ".pkl") 
  print "len(df_expansion): {}".format(len(df_expansions))
  

In [0]:
fig = plt.figure(figsize=(10, 7))
df_tmp = final_df[( final_df["fake" ] == 0 )]
plots = []
plots.append(plt.subplot(2,2,1))
cax = plt.scatter(df_tmp["euc_dist"], df_tmp[ "cos_dist"], c = [np.log(df_tmp[ "energy"].iloc[i]) for i in range(len(df_tmp[ "energy"]))], alpha = 0.05, cmap = 'autumn' )
plt.title("cos_dist FP")
cbar = fig.colorbar(cax)


plots.append(plt.subplot(2,2,3))
cax = plt.scatter(df_tmp["euc_dist"], df_tmp[ "err"], c =[np.log(df_tmp[ "energy"].iloc[i]) for i in range(len(df_tmp[ "energy"]))], alpha = 0.05, cmap = 'autumn' )
plt.title("euc_dist FP")
cbar = fig.colorbar(cax)

df_tmp = final_df[( final_df["fake" ] == 1 )]
plots.append(plt.subplot(2,2,2))
cax = plt.scatter(df_tmp["euc_dist"], df_tmp[ "cos_dist"], c =[np.log(df_tmp[ "energy"].iloc[i]) for i in range(len(df_tmp[ "energy"]))], alpha = 0.05, cmap = 'autumn' )
cbar = fig.colorbar(cax)

plt.title("cos_dist RAND")


plots.append(plt.subplot(2,2,4))
cax =  plt.scatter(df_tmp["euc_dist"], df_tmp[ "err"], c =[np.log(df_tmp[ "energy"].iloc[i]) for i in range(len(df_tmp[ "energy"]))], alpha = 0.05, cmap = 'autumn')
plt.title("euc_dist RAND")
cbar = fig.colorbar(cax)

#plt.savefig(path_res + "energy_and_fit_rnn_" +  FLAGS.source_rnn + "_num_" + str(num) + "_reg_" + reg + snapshop +  ".pdf")

for i, plot in enumerate(plots):
  plot.set_xlim([0,20])
  plot.set_ylim([0,(1 - i % 2)*1.4 + ((i % 2)) * 0.14])

In [0]:
#WE DONT HAVE A RESULT! FIXED POINTS DONT WORK!! LOOK AT BELOW: BLUE IS RANDOM, RED IS FP!!!  :( :( :( 
fig = plt.figure(figsize=(14, 7))

df_tmp = final_df
grouped = df_tmp.groupby([ 'fake', 'index'])
df_tmp['min_cos'] = grouped['cos_dist'].transform('min')
df_tmp['min_err'] = grouped['err'].transform('min')

plt.subplot(1,2,1)
plt.scatter(df_tmp[df_tmp["fake"] == 0]["euc_dist"], df_tmp[df_tmp["fake"] == 0][ "min_cos"], c ='r' ,hold=True,  alpha=0.05)
plt.scatter(df_tmp[df_tmp["fake"] == 1]["euc_dist"], df_tmp[df_tmp["fake"] == 1][ "min_cos"],  c='b' ,hold=True,  alpha=0.05)

plt.xlabel("euc-distance")
plt.ylabel("cosine distance")


plt.title("cos distance")

plt.subplot(1,2,2)
plt.scatter(df_tmp[df_tmp["fake"] == 0]["euc_dist"], df_tmp[df_tmp["fake"] == 0][ "min_err"], c ='r' ,hold=True,  alpha=0.05)
plt.scatter(df_tmp[df_tmp["fake"] == 1]["euc_dist"], df_tmp[df_tmp["fake"] == 1][ "min_err"],  c='b' ,hold=True,  alpha=0.05)

plt.title("err")

plt.xlabel("euc-distance")

plt.ylabel("rmsq error")



df_tmp.groupby([ 'fake'])['min_cos', 'min_err'].mean()


In [0]:
#loads the vanilla rnn model

#flags.DEFINE_integer("num_fp", 10, """Number of fixed points to run in parallel per input.""")
tf.reset_default_graph()
FLAGS.mode = 4
#FLAGS = flags.FLAGS
FLAGS.source_rnn = "rnn"
FLAGS.crazy_distance = False
FLAGS.num_fp = 10
FLAGS.num_fp_used = 5 #Warning: I set this to 5 for now to speed things up.
input_size = output_size = 27
reg = "0"
num = 40000
# root = "jfoerster21"
# new_nodes = 40

root = "jfoersternew22"

FLAGS.source_rnn = "rnn"
#FLAGS.source_rnn = "gru"
#rnn = FLAGS.source_rnn
rnn = "diff_rnn"  #WARNING !!!If you want to use the diff_rnn, uncomment this..!
rnn = "fasternn"
hpvalues = {}

batch_size = 1

rnn_cell_class = rnn_cell_variants.get_rnn_cell_class(rnn)
num_nodes, _ = rnn_cell_class.max_num_params_to_num_nodes(input_size, output_size, num, hpvalues, 1)

hparams= tf.HParams()
hparams.add_hparam('batch_size', batch_size)

rnn_cell_class.add_default_params(hparams)
tf.Variable(tf.zeros([batch_size, num_nodes]),
                          trainable=False)

rnn_cell = rnn_cell_class(input_size, num_nodes, output_size, hparams, layer = 'only')

input_placeholder = tf.placeholder(tf.float32, shape=[None, input_size])
state_placeholder = tf.placeholder(tf.float32, shape=[None, num_nodes])

output, hidden_state = rnn_cell(input_placeholder, state_placeholder)
#probs = tf.nn.softmax(output) #

#rnn_saver =  tf.train.Saver(tf.trainable_variables() )
for vr in tf.trainable_variables():
  shp = vr.get_shape()
  n_vr = int(np.prod(shp))
  print "param %s nels %d shape "%(vr.name, n_vr), shp

sess = tf.InteractiveSession()
# path = "/cns/ok-d/home/jfoerster/rnn_tune/" + root + "__rv" + FLAGS.source_rnn + "__tttext8__np"+str(num)+ \
#        "__n1_source" + FLAGS.source_rnn + "_l2_reg1/experiments_rnnfixed_point"
path = "/cns/ok-d/home/jfoerster/rnn_tune/jfoerster23__rv" + FLAGS.source_rnn + "__tttext8__np" + str(num) +"__n1_source" +FLAGS.source_rnn + "_l2_reg" + reg + "/experiments0_all" #This is a new run with simplified logic for hidden state

# path = "/cns/ok-d/home/jfoerster/rnn_tune/" + root + "rvdiff_rnn__tttext8__np"+str(num)+"__n1_sourcernn_l2_reg1/experiments_secondary" + str(FLAGS.mode)+"_all"
  
#rnn_weights_path = "/cns/ok-d/home/jfoerster/rnn_tune/jfoerster21__rv" + rnn + "__tttext8__np" + str(num) + "__n1_source" + rnn +  "_l2_reg0.1/experiments_rnndiff_rnn"

print path
rnn_saver.restore( sess, path)
fixed_points = sess.run(rnn_cell.expansion_points)



In [0]:


def get_min_dist(fps, h):
  min_dist = (np.sum(fps[0] - h_in)**2)**0.5
  indx = 0
  for ind, fp in enumerate( fps ):
    if ind < FLAGS.num_fp_used:
      d = (np.sum(fp - h_in)**2)**0.5
      if d < min_dist:
        min_dist = d
        indx = ind
  return min_dist, indx

task_class, _ = rnn_tuner.get_training_task_class("text8")
train_task = task_class("", batch_size, 100000,None, state_seed=1234, noise_seed=None)

In [0]:
#Compare quality of fit for random hidden states to random fixed points. Plot comparison by speed
input_placeholder = tf.placeholder(tf.float32, shape=[None, input_size])
state_placeholder = tf.placeholder(tf.float32, shape=[None, num_nodes])

output, hidden_state = rnn_cell(input_placeholder, state_placeholder)
probs = tf.nn.softmax(output)

print path
rnn_saver.restore( sess, path)

_= sess.run(rnn_cell.expansion_points.assign(fixed_points))

x, y = train_task.get_valid_next_batch()
print len(x), len(y)
print x[1].shape

h = np.zeros([batch_size, num_nodes])
#h  = sess.run(rnn_cell.zero_state)
df_fp = pd.DataFrame()
errors = []
for i in xrange(300):
  if i % 100 == 0:
    print i
  h_in = h
  t, h = sess.run([output, hidden_state], feed_dict={input_placeholder:x[i], state_placeholder:h})
  mse = np.mean(t**2)
  errors.append(t**2)
  #fixed_to_h = np.mean((fixed_points[0, np.argmax(x[i])] - h_in)**2)
  min_dist, indx = get_min_dist(fixed_points[:,np.argmax(x[i]), :], h_in)
  d = {"step":i, "diff":t, "h":h, "x":x[i], "y":y[i], "dist" :min_dist, "mse":mse, "ind" :indx}
  df_fp = df_fp.append(d, ignore_index=True)

print np.mean(errors)

#Pick random hidden states
copy_h = df["h"].values[:]
copy_h = [copy_h[i][0] for i in xrange(len(copy_h))]
copy_h = np.array(copy_h)

np.random.shuffle(copy_h)
random_h = copy_h[:270]
random_h = np.reshape(random_h, [10, 27, -1])
print random_h.shape

#Assign hidden states to the expansion point
_= sess.run(rnn_cell.expansion_points.assign(random_h))

h = np.zeros([batch_size, num_nodes]) 
df = pd.DataFrame()
errors = []
for i in xrange(300):
  if i % 100 == 0:
    print i
  h_in = h
  t, h = sess.run([output, hidden_state], feed_dict={input_placeholder:x[i], state_placeholder:h})
  mse = np.mean(t**2)
  errors.append(t**2)
  min_dist, indx = get_min_dist(random_h[:,np.argmax(x[i]), :], h_in)
  d = {"step":i, "diff":t, "h":h, "x":x[i], "y":y[i], "dist" :min_dist, "mse":mse, "ind" : indx}
  df = df.append(d, ignore_index=True)

print np.mean(errors)

inputs = random_h
#inputs = fixed_points
mode = 4
#fixed_points = sess.run(rnn_cell.expansion_points)
avgs = []
count = 0
for fixed_idx in xrange(len(fixed_points)):
  if fixed_idx != 0 and False:
    continue
  for char_idx in xrange(len(fixed_points[0])):
    fp = np.array([inputs[fixed_idx][char_idx]])
    char = np.array([np.eye(27)[char_idx]])
    #random.shuffle(fp[0])
    #fp = random_h[count]
    count+=1
    fp_image = sess.run([hidden_state], feed_dict={input_placeholder:char, state_placeholder:fp})
    avg = np.mean(np.abs(fp_image - fp))
    #print fixed_idx, char_idx, avg
    avgs.append(avg)
    
speed =  []
for i in ( df["ind"]):
  speed.append(avgs[np.int32(i)])
  
fig = plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
plt.scatter(df["dist"], df["mse"],  c = speed)
plt.title("random points")
print( np.corrcoef(df["dist"], df["mse"]))

plt.subplot(1, 2, 2)
plt.plot(df_fp["dist"], df_fp["mse"], "r.")
plt.title("fixed points")
print( np.corrcoef(df_fp["dist"], df_fp["mse"]))

In [0]:
from sklearn.manifold import TSNE
def plot_tsne(data_points, labels):
  
  model = TSNE(n_components=2, random_state=0)
  np.set_printoptions(suppress=True)
  fit_tsne = model.fit_transform(data_points)

  def map_vals(vals):
    vals_set = list(set(vals))
    map_dict = {}
    for i, v in enumerate(vals_set):
      map_dict[v] = i
    return [map_dict[v] for v in vals]

  print fit_tsne

  x = [h[0] for h in h_tnse]
  y = [h[1] for h in h_tnse]

  colors = map_vals(labels)
  print len(set(colors)) 

  p = plt.scatter(x,y,c = colors, cmap = "Set1")

In [0]:
print w_out.shape
print w_x.shape
print b_ixn.shape
print out_bias.shape

In [0]:
np.corrcoef(np.zeros((10,100)), rowvar=False).shape

In [0]:
top_10_ev = []
top_10_el = []

top_1_v = []

top_10_v = []
top_10_u = []

plt.figure(figsize=(15,10), dpi=300)
for i,c in enumerate(mapping):
  plt.subplot(5,6,i+1)
  char = mapping[c]

  el, ev = np.linalg.eig(w_x[char])
  top_10_ev.append(ev[:, :10])
  top_10_el.append(el[:10])
  
  u,s,v = np.linalg.svd(w_x[char])
  
  top_1_v.append(v[:1,:].T)
  top_10_v.append(v[:10, :].T)
  top_10_u.append(u[:,:10])
  
  #plt.imshow(np.abs(np.corrcoef(np.abs(v[:, :50]), rowvar=False,)), 
  #           'Greys_r', interpolation='nearest')
  #plt.colorbar()
  plt.stem(s)
  
  
  plt.title(c)

In [0]:
np.linalg.svd?

In [0]:
plt.imshow(cd(v[:10,: ].T, w_out), interpolation='nearest')
plt.grid(False)
plt.colorbar()

In [0]:
def cd(u,v):
  u = u / np.sqrt((u**2).sum(0, keepdims=True))
  v = v / np.sqrt((v**2).sum(0, keepdims=True))
  return 1-u.T.dot(v)

In [0]:
np.max(np.abs((w_x[char] - u.dot(np.diag(s)).dot(v))))

In [0]:
spc_fix_pt = np.linalg.inv(np.eye(216) - w_x[mapping[' ']]).dot(b_ixn[mapping[' ']])
spc_fix_pt = b_ixn[mapping[' ']]
plt.scatter(spc_fix_pt, top_10_v[1][:,0])

np.corrcoef(spc_fix_pt, top_10_v[1][:,0])




In [0]:
plt.figure(figsize=(15,15))

k = 5
top_10_v_s = np.hstack([m[:,:k] for m in top_10_v])
print top_10_v_s.shape
top_10_u_s = np.hstack([m[:,:k] for m in top_10_u])
print top_10_u_s.shape

vecs = np.hstack((top_10_v_s, top_10_u_s, b_ixn.T, w_out))
vecs.shape

plt.imshow(np.abs(1 - cd(vecs,vecs)), 'Greys_r', interpolation='nearest')
plt.colorbar()

plt.yticks(np.arange(k*0.5 - 0.5, 27*k ,k), [pretty_chars[i] for i in xrange(27)])
plt.xticks(np.arange(k*0.5 - 0.5, 27*k ,k), [pretty_chars[i] for i in xrange(27)])
plt.grid(False)
None

In [0]:
plt.figure(figsize=(10,10))

k = 1
top_10_v_s = np.hstack([m[:,:k] for m in top_10_v])
print top_10_v_s.shape
top_10_u_s = np.hstack([m[:,:k] for m in top_10_u])
print top_10_u_s.shape

plt.imshow(np.abs(np.corrcoef(np.hstack((top_10_v_s, top_10_u_s, b_ixn.T, w_out)), rowvar=False)), 'Greys_r', interpolation='nearest')
plt.colorbar()

plt.yticks(np.arange(k*0.5 - 1, 27*k ,k), [pretty_chars[i] for i in xrange(27)])
plt.xticks(np.arange(k*0.5 - 1, 27*k ,k), [pretty_chars[i] for i in xrange(27)])
plt.grid(False)
None

In [0]:
plt.plot(np.abs(ev[:,:10].T.dot(w_out).T))
plt.vlines(char, *plt.ylim())

In [0]:
top_10_ev = []
top_10_el = []

top_1_v = []

top_10_v = []
top_10_u = []

plt.figure(figsize=(15,10), dpi=300)
for i,c in enumerate(mapping):
  plt.subplot(5,6,i+1)
  char = mapping[c]

  el, ev = np.linalg.eig(w_x[char])
  top_10_ev.append(ev[:, :10])
  top_10_el.append(el[:10])
  
  u,s,v = np.linalg.svd(w_x[char])
  
  top_1_v.append(v[:,:1])
  top_10_v.append(v[:,:10])
  top_10_u.append(u[:,:10])
  
  #plt.imshow(np.abs(np.corrcoef(np.abs(v[:, :50]), rowvar=False,)), 
  #           'Greys_r', interpolation='nearest')
  #plt.colorbar()
  plt.stem(s)
  
  
  plt.title(c)

In [0]:
string = " the long and winding text and some more of it improbable "

hs = [h_zero.T]

for char in string:
  hs.append(make_step(hs[-1], mapping[char])[0])

hs = np.hstack(hs[1:])

kk=1
V = np.hstack([m[:,:1] for m in top_10_v[kk:kk+1]])
U = np.hstack([m[:,:1] for m in top_10_u[:]])

if 0:
  plt.imshow(V.T.dot(hs), interpolation='nearest', cmap='jet')
  plt.xticks(*zip(*enumerate(string)))
  plt.grid(False)
  plt.axis('tight')
  
#plt.plot(V.T.dot(hs)[0,:])
#plt.plot((w_out.T.dot(hs) + out_bias[:,None])[0]*-0.13)

plt.scatter(V.T.dot(hs), (w_out.T.dot(hs) + out_bias[:,None])[0])

#plt.xticks(*zip(*enumerate(string)))
#plt.grid(False)
None

In [0]:
plt.plot(U[:,:2].T.dot(w_out).T)

In [0]:
plt.plot(U[:,:2].T.dot(w_out).T)

In [0]:
-

u_1, s_1, v_1 = np.linalg.svd( w_x[mapping['t']] )
u_2, s_2, v_2 = np.linalg.svd( w_x[mapping['h']] )


plt.figure()
mmm = v_2.dot(u_1)
mmm = np.diag(np.sqrt(s_2)).dot(mmm.dot(np.diag(np.sqrt(s_1))))
plt.imshow(np.abs(mmm), 'Greys')
plt.grid(False)
plt.colorbar()

plt.figure()
plt.hist((mmm).ravel(), bins=250)
#plt.ylim(0, 200)

(np.abs(mmm)>1e-1).sum()